In [15]:
import pandas as pd 
from bs4 import BeautifulSoup  
from selenium.webdriver.common.by import By
import time 
import numpy as np
import re
import json
from seleniumwire import webdriver
from seleniumwire.utils import decode

# Seed list

In [2]:
data = pd.read_excel('Seedlist.xlsx')

In [3]:
data

,name,rating,ratings,app_store,review_1,review_2,uk_app,category
0,YNAB: Budgeting & Finance,4.8,46400,https://apps.apple.com/us/app/ynab-you-need-a-...,https://appletoolbox.com/the-best-iphone-budge...,NaN,NaN,Budgeting & money management
1,Daily Budget Original,4.7,4800,https://apps.apple.com/us/app/daily-budget-ori...,https://appletoolbox.com/the-best-iphone-budge...,NaN,NaN,Budgeting & money management
2,Mint: Budget & Expense Manager,4.8,783700,https://apps.apple.com/us/app/mint-budget-expe...,https://appletoolbox.com/the-best-iphone-budge...,NaN,NaN,Budgeting & money management
3,EveryDollar: Budget Your Money,4.7,61500,https://apps.apple.com/us/app/everydollar-budg...,https://appletoolbox.com/the-best-iphone-budge...,NaN,NaN,Budgeting & money management
4,Goodbudget Budget Planner,4.7,12800,https://apps.apple.com/us/app/goodbudget-budge...,https://appletoolbox.com/the-best-iphone-budge...,NaN,NaN,Budgeting & money management
...,...,...,...,...,...,...,...,...
199,Nelo,4.4,159,https://apps.apple.com/us/app/nelo/id1477987798,NaN,NaN,NaN,"Buy now, pay later"
200,Zebit,3.3,78,https://apps.apple.com/us/app/zebit/id1574137193,NaN,NaN,NaN,"Buy now, pay later"
201,Split,3.7,35,https://apps.apple.com/us/app/split-enjoy-now-...,NaN,NaN,NaN,"Buy now, pay later"
202,LazyPay: Loan App & Pay Later,4.8,116.1K,https://apps.apple.com/in/app/lazypay-loan-app...,https://www.mobileappdaily.com/products/best-b...,NaN,NaN,"Buy now, pay later"


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       204 non-null    object 
 1   rating     200 non-null    float64
 2   ratings    200 non-null    object 
 3   app_store  204 non-null    object 
 4   review_1   155 non-null    object 
 5   review_2   9 non-null      object 
 6   uk_app     28 non-null     object 
 7   category   204 non-null    object 
dtypes: float64(1), object(7)
memory usage: 12.9+ KB


In [485]:
data.category.unique()

array(['Budgeting & money management', 'Stock trading & investing',
       'Cash back apps', 'Coupon apps', 'Money transfer apps',
       'Online banking apps', 'Buy now, pay later'], dtype=object)

# Scrape data

In [99]:
#df = pd.DataFrame(columns=['name', 'url', 'stars', 'ratings',  
#                           'category', 'scraped_rating', 'json'])

In [100]:
#parse_again = pd.DataFrame()

In [101]:
for index, row in data.iterrows():
    
    url = row['app_store']
    
    try:
        driver = webdriver.Chrome() 
        driver.get(url)
        time.sleep(max(5.564878, 6.091354 + 2*np.random.randn()))

        button = driver.find_element(By.XPATH, "//button[contains(text(),'See Details') and contains(@class, 'we-modal__show link section__nav__see-all-link')]")
        button.click()
        time.sleep(max(5.564878, 6.091354 + 2*np.random.randn()))

        soup  = BeautifulSoup(driver.page_source)
        rating_count = soup.find_all("figcaption", 'we-rating-count star-rating__count')[0].text

        for request in driver.requests:
            if request.response:
                if request.url.startswith("https://amp-api.apps.apple.com/v1/catalog/"):
                    body = decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity'))
                    body_str = body.decode('utf-8')  # Decode the bytes to a string
                    json_data = json.loads(body_str)  # Parse the string as JSON


        # Create a DataFrame for the current iteration
        current_data = pd.DataFrame({'name': [row['name']],
                                     'url': [url],
                                     'stars': [row['rating']],
                                     'ratings': [row['ratings']],
                                     'category': [row['category']],  
                                     'scraped_rating': [rating_count],
                                     'json': [json_data]})


        df = pd.concat([df, current_data], ignore_index=True)
        driver.close()
        driver.quit()
        print(f"Row {index}: Done")
        print()
        
    except Exception as e:
        print(f"Failed to parse row {index}")
        print(url)
        print()
        parse_again = parse_again.append(row, ignore_index=True) 
        

Row 0: Done

Row 1: Done

Row 2: Done

Row 3: Done

Row 4: Done

Row 5: Done

Row 6: Done

Row 7: Done

Row 8: Done

Row 9: Done

Row 10: Done

Row 11: Done

Row 12: Done

Row 13: Done

Row 14: Done

Row 15: Done

Row 16: Done

Row 17: Done

Row 18: Done

Row 19: Done

Row 20: Done

Row 21: Done

Row 22: Done

Row 23: Done

Row 24: Done

Row 25: Done

Row 26: Done

Row 27: Done

Row 28: Done

Row 29: Done

Row 30: Done

Row 31: Done

Row 32: Done

Row 33: Done

Row 34: Done

Row 35: Done

Row 36: Done

Row 37: Done

Row 38: Done

Row 39: Done

Row 40: Done

Row 41: Done

Row 42: Done

Row 43: Done

Row 44: Done

Row 45: Done

Row 46: Done

Failed to parse row 47
https://apps.apple.com/es/app/tickmill/id1548644102

Row 48: Done

Row 49: Done

Failed to parse row 50
https://apps.apple.com/me/app/fxtm-trader-forex-trading/id1265719831

Row 51: Done

Row 52: Done

Row 53: Done

Row 54: Done

Row 55: Done

Row 56: Done

Row 57: Done

Row 58: Done

Row 59: Done

Row 60: Done

Row 61: Done

R

Exception in thread Http2SingleStreamLayer-9:
Traceback (most recent call last):
  File "/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/seleniumwire/thirdparty/mitmproxy/server/protocol/http2.py", line 719, in run
    layer()
  File "/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/seleniumwire/thirdparty/mitmproxy/server/protocol/http.py", line 206, in __call__
    if not self._process_flow(flow):
  File "/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/seleniumwire/thirdparty/mitmproxy/server/protocol/http.py", line 388, in _process_flow
    get_response()
  File "/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/seleniumwire/thirdparty/mitmproxy/server/protocol/http.py", line 381, in get_response
    self.send_request_body(f.request, [f.request.data.content]

Row 143: Done

Row 144: Done

Row 145: Done

Row 146: Done

Row 147: Done

Row 148: Done

Row 149: Done

Row 150: Done

Row 151: Done

Row 152: Done

Row 153: Done

Row 154: Done

Row 155: Done

Row 156: Done

Row 157: Done

Row 158: Done

Row 159: Done

Row 160: Done

Row 161: Done

Row 162: Done

Row 163: Done

Row 164: Done

Row 165: Done

Row 166: Done

Row 167: Done

Row 168: Done

Row 169: Done

Row 170: Done

Row 171: Done

Row 172: Done

Row 173: Done

Row 174: Done

Row 175: Done



/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/asyncio/base_events.py:612: RuntimeWarning: coroutine 'AddonManager.handle_lifecycle' was never awaited
  self._ready.clear()


Row 176: Done

Row 177: Done

Row 178: Done

Row 179: Done

Failed to parse row 180
https://www.apple.com/apple-cash/

Row 181: Done

Row 182: Done

Row 183: Done

Row 184: Done

Row 185: Done

Row 186: Done

Row 187: Done

Row 188: Done

Row 189: Done

Row 190: Done

Row 191: Done

Row 192: Done

Row 193: Done

Row 194: Done

Row 195: Done

Row 196: Done

Row 197: Done

Row 198: Done

Row 199: Done

Row 200: Done

Row 201: Done

Row 202: Done

Row 203: Done



In [102]:
df

,name,url,stars,ratings,category,scraped_rating,json
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',..."
1,Daily Budget Original,https://apps.apple.com/us/app/daily-budget-ori...,4.7,4800,Budgeting & money management,4.7 • 4.8K Ratings,"{'data': [{'id': '651896614', 'type': 'apps', ..."
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ..."
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ..."
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ..."
...,...,...,...,...,...,...,...
194,Nelo,https://apps.apple.com/us/app/nelo/id1477987798,4.4,159,"Buy now, pay later",4.4 • 159 Ratings,"{'data': [{'id': '1477987798', 'type': 'apps',..."
195,Zebit,https://apps.apple.com/us/app/zebit/id1574137193,3.3,78,"Buy now, pay later",3.3 • 78 Ratings,"{'data': [{'id': '1477987798', 'type': 'apps',..."
196,Split,https://apps.apple.com/us/app/split-enjoy-now-...,3.7,35,"Buy now, pay later",3.6 • 36 Ratings,"{'data': [{'id': '1477987798', 'type': 'apps',..."
197,LazyPay: Loan App & Pay Later,https://apps.apple.com/in/app/lazypay-loan-app...,4.8,116.1K,"Buy now, pay later",4.8 • 116.7K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',..."


In [103]:
#df.to_csv('raw_data.csv')

# Clean data, feature engineering

In [426]:
df = pd.read_csv('raw_data.csv', index_col=0)

There are 5 missing rows, manually add them

In [123]:
missing_data = pd.read_excel('missing_data.xlsx', index_col=0)

In [124]:
missing_data['json'] = missing_data['json'].apply(lambda x: x.replace('\n', '') if pd.notna(x) else x)

In [125]:
missing_data

,name,url,stars,ratings,category,scraped_rating,json
0,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{ ""data"": [ { ""id"": ""1548..."
1,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{ ""data"": [ { ""id"": ""1265..."
2,Freebird: Finance Manager Pro,https://apps.apple.com/gb/app/free-bird-simple...,4.3,9,Cash back apps,4.3 • 9,"{ ""data"": [ { ""id"": ""1244..."
3,Apple Cash,https://support.apple.com/en-us/HT207886,0.0,0,Money transfer apps,0,NaN
4,Apple Cash / Pay later,https://www.apple.com/apple-cash/,0.0,0,"Buy now, pay later",0,NaN


In [427]:
df = pd.concat([df, missing_data], ignore_index=True)

In [428]:
df

,name,url,stars,ratings,category,scraped_rating,json
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',..."
1,Daily Budget Original,https://apps.apple.com/us/app/daily-budget-ori...,4.7,4800,Budgeting & money management,4.7 • 4.8K Ratings,"{'data': [{'id': '651896614', 'type': 'apps', ..."
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ..."
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ..."
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ..."
...,...,...,...,...,...,...,...
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{ ""data"": [ { ""id"": ""1548..."
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{ ""data"": [ { ""id"": ""1265..."
201,Freebird: Finance Manager Pro,https://apps.apple.com/gb/app/free-bird-simple...,4.3,9,Cash back apps,4.3 • 9,"{ ""data"": [ { ""id"": ""1244..."
202,Apple Cash,https://support.apple.com/en-us/HT207886,0.0,0,Money transfer apps,0,NaN


In [429]:
df['json'] = df['json'].apply(lambda x: x.replace('\'', '\"').replace('None', 'null').replace('\\xa0', '') if pd.notna(x) else x)

In [430]:
json_199 = '''{
  "data": [
    {
      "id": "1548644102",
      "type": "apps",
      "href": "/v1/catalog/es/apps/1548644102?l=es-ES",
      "attributes": {
        "privacyDetails": {
          "managePrivacyChoicesUrl": null,
          "privacyTypes": [
            {
              "privacyType": "Datos usados para rastrearte",
              "identifier": "DATA_USED_TO_TRACK_YOU",
              "description": "Los siguientes datos pueden usarse para rastrearte en apps y sitios web propiedad de otras empresas:",
              "dataCategories": [
                {
                  "dataCategory": "IdentificadoADres",
                  "identifier": "IDENTIFIERS",
                  "dataTypes": [
                    "ID de usuario",
                    "ID del dispositivo"
                  ]
                },
                {
                  "dataCategory": "Datos de uso",
                  "identifier": "USAGE_DATA",
                  "dataTypes": [
                    "Interacción con el producto"
                  ]
                },
                {
                  "dataCategory": "Diagnósticos",
                  "identifier": "DIAGNOSTICS",
                  "dataTypes": [
                    "Datos de rendimiento"
                  ]
                },
                {
                  "dataCategory": "Otros datos",
                  "identifier": "OTHER",
                  "dataTypes": [
                    "Otros tipos de datos"
                  ]
                }
              ],
              "purposes": []
            },
            {
              "privacyType": "Datos vinculados contigo",
              "identifier": "DATA_LINKED_TO_YOU",
              "description": "Los siguientes datos, que pueden recopilarse y vincularse con tu identidad, pueden utilizarse para los fines siguientes:",
              "dataCategories": [],
              "purposes": [
                {
                  "purpose": "Análisis de datos",
                  "identifier": "ANALYTICS",
                  "dataCategories": [
                    {
                      "dataCategory": "IdentificadoADres",
                      "identifier": "IDENTIFIERS",
                      "dataTypes": [
                        "ID de usuario",
                        "ID del dispositivo"
                      ]
                    },
                    {
                      "dataCategory": "Datos de uso",
                      "identifier": "USAGE_DATA",
                      "dataTypes": [
                        "Interacción con el producto"
                      ]
                    },
                    {
                      "dataCategory": "Diagnósticos",
                      "identifier": "DIAGNOSTICS",
                      "dataTypes": [
                        "Datos de rendimiento"
                      ]
                    },
                    {
                      "dataCategory": "Otros datos",
                      "identifier": "OTHER",
                      "dataTypes": [
                        "Otros tipos de datos"
                      ]
                    }
                  ]
                },
                {
                  "purpose": "Funcionalidad de la app",
                  "identifier": "APP_FUNCTIONALITY",
                  "dataCategories": [
                    {
                      "dataCategory": "Información financiera",
                      "identifier": "FINANCIAL_INFO",
                      "dataTypes": [
                        "Información de pago",
                        "Otra información financiera"
                      ]
                    },
                    {
                      "dataCategory": "Datos de contacto",
                      "identifier": "CONTACT_INFO",
                      "dataTypes": [
                        "Dirección física",
                        "Dirección de correo electrónico",
                        "Nombre",
                        "Número de teléfono"
                      ]
                    },
                    {
                      "dataCategory": "Contenido del usuario",
                      "identifier": "USER_CONTENT",
                      "dataTypes": [
                        "Fotos o vídeos",
                        "Atención al cliente"
                      ]
                    },
                    {
                      "dataCategory": "IdentificadoADres",
                      "identifier": "IDENTIFIERS",
                      "dataTypes": [
                        "ID de usuario",
                        "ID del dispositivo"
                      ]
                    },
                    {
                      "dataCategory": "Datos de uso",
                      "identifier": "USAGE_DATA",
                      "dataTypes": [
                        "Interacción con el producto"
                      ]
                    },
                    {
                      "dataCategory": "Datos sensibles",
                      "identifier": "SENSITIVE_INFO",
                      "dataTypes": [
                        "Datos sensibles"
                      ]
                    },
                    {
                      "dataCategory": "Diagnósticos",
                      "identifier": "DIAGNOSTICS",
                      "dataTypes": [
                        "Datos de errores",
                        "Datos de rendimiento"
                      ]
                    },
                    {
                      "dataCategory": "Otros datos",
                      "identifier": "OTHER",
                      "dataTypes": [
                        "Otros tipos de datos"
                      ]
                    }
                  ]
                }
              ]
            }
          ]
        }
      },
      "relationships": {
        "developer": {
          "href": "/v1/catalog/es/apps/1548644102/developer?l=es-ES",
          "data": [
            {
              "id": "1548644104",
              "type": "developers",
              "href": "/v1/catalog/es/developers/1548644104?l=es-ES"
            }
          ]
        },
        "genres": {
          "href": "/v1/catalog/es/apps/1548644102/genres?l=es-ES",
          "data": [
            {
              "id": "6015",
              "type": "genres",
              "href": "/v1/catalog/es/genres/6015?l=es-ES"
            }
          ]
        }
      },
      "meta": {
        "contentVersion": {
          "MZ_INDEXER": 1693558752688,
          "RTCI": 0
        }
      }
    }
  ]
}
'''

In [431]:
df.loc[199, 'json'] = json_199

In [432]:
df['json'] = df['json'].apply(lambda x: json.loads(x) if pd.notna(x) else x)

In [433]:
df

,name,url,stars,ratings,category,scraped_rating,json
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',..."
1,Daily Budget Original,https://apps.apple.com/us/app/daily-budget-ori...,4.7,4800,Budgeting & money management,4.7 • 4.8K Ratings,"{'data': [{'id': '651896614', 'type': 'apps', ..."
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ..."
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ..."
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ..."
...,...,...,...,...,...,...,...
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1548644102', 'type': 'apps',..."
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',..."
201,Freebird: Finance Manager Pro,https://apps.apple.com/gb/app/free-bird-simple...,4.3,9,Cash back apps,4.3 • 9,"{'data': [{'id': '1244075045', 'type': 'apps',..."
202,Apple Cash,https://support.apple.com/en-us/HT207886,0.0,0,Money transfer apps,0,NaN


In [434]:
def extract_privacy_details(json_data):
    if json_data is None:
        return None
    return json_data.get('data')[0].get('attributes').get('privacyDetails').get('privacyTypes')

In [435]:
df['privacyDetails'] = df['json'][:202].apply(extract_privacy_details)

In [436]:
df

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi..."
1,Daily Budget Original,https://apps.apple.com/us/app/daily-budget-ori...,4.7,4800,Budgeting & money management,4.7 • 4.8K Ratings,"{'data': [{'id': '651896614', 'type': 'apps', ...","[{'privacyType': 'Data Not Collected', 'identi..."
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id..."
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id..."
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi..."
...,...,...,...,...,...,...,...,...
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1548644102', 'type': 'apps',...",[{'privacyType': 'Datos usados para rastrearte...
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id..."
201,Freebird: Finance Manager Pro,https://apps.apple.com/gb/app/free-bird-simple...,4.3,9,Cash back apps,4.3 • 9,"{'data': [{'id': '1244075045', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi..."
202,Apple Cash,https://support.apple.com/en-us/HT207886,0.0,0,Money transfer apps,0,NaN,NaN


In [437]:
def extract_DATA_LINKED_TO_YOU(json_data):
    for data_dict in json_data:
        if data_dict['identifier'] == 'DATA_LINKED_TO_YOU':
            desired_data = data_dict
            break
        else:
            desired_data = 'DATA_NOT_COLLECTED'
    return desired_data

In [438]:
df['DATA_LINKED_TO_YOU'] = df['privacyDetails'][:202].apply(extract_DATA_LINKED_TO_YOU)

In [439]:
df

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif..."
1,Daily Budget Original,https://apps.apple.com/us/app/daily-budget-ori...,4.7,4800,Budgeting & money management,4.7 • 4.8K Ratings,"{'data': [{'id': '651896614', 'type': 'apps', ...","[{'privacyType': 'Data Not Collected', 'identi...",DATA_NOT_COLLECTED
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif..."
...,...,...,...,...,...,...,...,...,...
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1548644102', 'type': 'apps',...",[{'privacyType': 'Datos usados para rastrearte...,"{'privacyType': 'Datos vinculados contigo', 'i..."
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."
201,Freebird: Finance Manager Pro,https://apps.apple.com/gb/app/free-bird-simple...,4.3,9,Cash back apps,4.3 • 9,"{'data': [{'id': '1244075045', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif..."
202,Apple Cash,https://support.apple.com/en-us/HT207886,0.0,0,Money transfer apps,0,NaN,NaN,NaN


In [440]:
df.loc[202, 'DATA_LINKED_TO_YOU'] = 'DATA_NOT_COLLECTED'
df.loc[203, 'DATA_LINKED_TO_YOU'] = 'DATA_NOT_COLLECTED'

In [444]:
# Money manager, expense tracker
dict_23 = '{"privacyType":"Data Linked to You","identifier":"DATA_LINKED_TO_YOU","description":"The following data, which may be collected and linked to your identity, may be used for the following purposes:","dataCategories":[],"purposes":[{"purpose":"Third-Party Advertising","identifier":"THIRD_PARTY_ADVERTISING","dataCategories":[{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["Device ID"]}]},{"purpose":"Analytics","identifier":"ANALYTICS","dataCategories":[{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID","Device ID"]},{"dataCategory":"Usage Data","identifier":"USAGE_DATA","dataTypes":["Product Interaction","Other Usage Data"]},{"dataCategory":"Diagnostics","identifier":"DIAGNOSTICS","dataTypes":["Crash Data","Other Diagnostic Data"]}]},{"purpose":"Product Personalization","identifier":"PRODUCT_PERSONALIZATION","dataCategories":[{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID"]},{"dataCategory":"Usage Data","identifier":"USAGE_DATA","dataTypes":["Product Interaction"]}]},{"purpose":"App Functionality","identifier":"APP_FUNCTIONALITY","dataCategories":[{"dataCategory":"Contact Info","identifier":"CONTACT_INFO","dataTypes":["Email Address","Name"]},{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID","Device ID"]},{"dataCategory":"Usage Data","identifier":"USAGE_DATA","dataTypes":["Product Interaction","Other Usage Data"]},{"dataCategory":"Diagnostics","identifier":"DIAGNOSTICS","dataTypes":["Crash Data","Other Diagnostic Data"]}]}]}'

In [446]:
df.loc[23, 'DATA_LINKED_TO_YOU'] = dict_23

In [447]:
# Wallet - Daily Budget & Profit
dict_25 = '{"privacyType":"Data Linked to You","identifier":"DATA_LINKED_TO_YOU","description":"The following data, which may be collected and linked to your identity, may be used for the following purposes:","dataCategories":[],"purposes":[{"purpose":"Developer’s Advertising or Marketing","identifier":"DEVELOPERS_ADVERTISING","dataCategories":[{"dataCategory":"Contact Info","identifier":"CONTACT_INFO","dataTypes":["Email Address","Name"]},{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID"]},{"dataCategory":"Usage Data","identifier":"USAGE_DATA","dataTypes":["Product Interaction"]}]},{"purpose":"Analytics","identifier":"ANALYTICS","dataCategories":[{"dataCategory":"Contact Info","identifier":"CONTACT_INFO","dataTypes":["Email Address","Name"]},{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID","Device ID"]},{"dataCategory":"Usage Data","identifier":"USAGE_DATA","dataTypes":["Product Interaction"]}]},{"purpose":"Product Personalization","identifier":"PRODUCT_PERSONALIZATION","dataCategories":[{"dataCategory":"Usage Data","identifier":"USAGE_DATA","dataTypes":["Product Interaction"]}]},{"purpose":"App Functionality","identifier":"APP_FUNCTIONALITY","dataCategories":[{"dataCategory":"Contact Info","identifier":"CONTACT_INFO","dataTypes":["Email Address"]},{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID"]}]}]}'

In [448]:
df.loc[25, 'DATA_LINKED_TO_YOU'] = dict_25

In [449]:
# BOH Mobile Banking
dict_150 = '{"privacyType":"Data Linked to You","identifier":"DATA_LINKED_TO_YOU","description":"The following data, which may be collected and linked to your identity, may be used for the following purposes:","dataCategories":[],"purposes":[{"purpose":"Analytics","identifier":"ANALYTICS","dataCategories":[{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID","Device ID"]},{"dataCategory":"Diagnostics","identifier":"DIAGNOSTICS","dataTypes":["Other Diagnostic Data"]}]},{"purpose":"App Functionality","identifier":"APP_FUNCTIONALITY","dataCategories":[{"dataCategory":"Location","identifier":"LOCATION","dataTypes":["Precise Location","Coarse Location"]},{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID","Device ID"]},{"dataCategory":"Diagnostics","identifier":"DIAGNOSTICS","dataTypes":["Crash Data","Performance Data"]}]},{"purpose":"Other Purposes","identifier":"OTHER_PURPOSES","dataCategories":[{"dataCategory":"Location","identifier":"LOCATION","dataTypes":["Precise Location","Coarse Location"]},{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID","Device ID"]},{"dataCategory":"Diagnostics","identifier":"DIAGNOSTICS","dataTypes":["Crash Data","Performance Data","Other Diagnostic Data"]}]}]}'

In [450]:
df.loc[150, 'DATA_LINKED_TO_YOU'] = dict_150

In [451]:
# T-Mobile MONEY
dict_151 = '{"privacyType":"Data Linked to You","identifier":"DATA_LINKED_TO_YOU","description":"The following data, which may be collected and linked to your identity, may be used for the following purposes:","dataCategories":[],"purposes":[{"purpose":"App Functionality","identifier":"APP_FUNCTIONALITY","dataCategories":[{"dataCategory":"Financial Info","identifier":"FINANCIAL_INFO","dataTypes":["Payment Info"]},{"dataCategory":"Location","identifier":"LOCATION","dataTypes":["Precise Location"]},{"dataCategory":"Contact Info","identifier":"CONTACT_INFO","dataTypes":["Physical Address","Email Address","Name","Phone Number","Other User Contact Info"]},{"dataCategory":"User Content","identifier":"USER_CONTENT","dataTypes":["Photos or Videos","Customer Support"]},{"dataCategory":"Identifiers","identifier":"IDENTIFIERS","dataTypes":["User ID","Device ID"]},{"dataCategory":"Usage Data","identifier":"USAGE_DATA","dataTypes":["Product Interaction"]}]}]}'

In [452]:
df.loc[151, 'DATA_LINKED_TO_YOU'] = dict_151

In [460]:
# Function to safely load JSON
def safe_json_loads(x):
    try:
        return json.loads(x) if isinstance(x, str) else x
    except (ValueError, json.JSONDecodeError):
        return None  # Return None for invalid JSON strings or other types

# Apply the safe_json_loads function to the column
df['DATA_LINKED_TO_YOU'] = df['DATA_LINKED_TO_YOU'].apply(safe_json_loads)

In [465]:
df['DATA_LINKED_TO_YOU'] = df['DATA_LINKED_TO_YOU'].fillna('DATA_NOT_COLLECTED')

In [466]:
len(df[df.DATA_LINKED_TO_YOU=='DATA_NOT_COLLECTED'])

12

In [467]:
df[df.DATA_LINKED_TO_YOU=='DATA_NOT_COLLECTED']

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU
1,Daily Budget Original,https://apps.apple.com/us/app/daily-budget-ori...,4.7,4800,Budgeting & money management,4.7 • 4.8K Ratings,"{'data': [{'id': '651896614', 'type': 'apps', ...","[{'privacyType': 'Data Not Collected', 'identi...",DATA_NOT_COLLECTED
11,Fudget 2: Budget Planner,https://apps.apple.com/us/app/fudget-2-budget-...,4.0,28,Budgeting & money management,4.1 • 34 Ratings,"{'data': [{'id': '1664719450', 'type': 'apps',...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED
21,Money Manager Expense & Budget,https://apps.apple.com/us/app/money-manager-ex...,4.8,9500,Budgeting & money management,4.8 • 9.5K Ratings,"{'data': [{'id': '560481810', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...",DATA_NOT_COLLECTED
22,"Expense Manager, Money Tracker",https://apps.apple.com/us/app/expense-manager-...,4.7,846,Budgeting & money management,4.7 • 847 Ratings,"{'data': [{'id': '560481810', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...",DATA_NOT_COLLECTED
24,Monefy: Money Tracker,https://apps.apple.com/us/app/monefy-money-tra...,4.7,4400,Budgeting & money management,4.7 • 4.5K Ratings,"{'data': [{'id': '1212024409', 'type': 'apps',...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED
129,Alliant Mobile Banking,https://apps.apple.com/us/app/alliant-mobile-b...,4.7,8.1K,Online banking apps,4.7 • 8.1K Ratings,"{'data': [{'id': '407107217', 'type': 'apps', ...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED
136,TD Bank (US),https://apps.apple.com/us/app/td-bank-us/id382...,4.8,246.5K,Online banking apps,4.8 • 246.5K Ratings,"{'data': [{'id': '382107453', 'type': 'apps', ...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED
149,Zopa Bank,https://apps.apple.com/gb/app/zopa-bank/id1324...,4.9,20400,Online banking apps,4.9 • 21.1K Ratings,"{'data': [{'id': '1324011914', 'type': 'apps',...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED
169,PenFed Mobile,https://apps.apple.com/us/app/penfed-mobile/id...,4.2,9600,Online banking apps,4.2 • 9.6K Ratings,"{'data': [{'id': '460008350', 'type': 'apps', ...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED
172,Navy Federal Credit Union,https://apps.apple.com/us/app/navy-federal-cre...,4.8,569.5K,Online banking apps,4.9 • 575.4K Ratings,"{'data': [{'id': '370811491', 'type': 'apps', ...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED


In [468]:
to_add = df[df.DATA_LINKED_TO_YOU =='DATA_NOT_COLLECTED']

In [469]:
df = df[df.DATA_LINKED_TO_YOU !='DATA_NOT_COLLECTED']

In [470]:
df

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif..."
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif..."
5,Wally: Smart Personal Finance,https://apps.apple.com/us/app/wally-smart-pers...,3.8,1600,Budgeting & money management,3.8 • 1.6K Ratings,"{'data': [{'id': '1178011327', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."
...,...,...,...,...,...,...,...,...,...
197,LazyPay: Loan App & Pay Later,https://apps.apple.com/in/app/lazypay-loan-app...,4.8,116.1K,"Buy now, pay later",4.8 • 116.7K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."
198,Zilch: Pay over time,https://apps.apple.com/gb/app/zilch-pay-over-t...,4.2,2.1K,"Buy now, pay later",4.2 • 2.1K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1548644102', 'type': 'apps',...",[{'privacyType': 'Datos usados para rastrearte...,"{'privacyType': 'Datos vinculados contigo', 'i..."
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif..."


In [371]:
def extract_purposes(json_data):
    
    category_list = []
    purposes = json_data.get('purposes')
    
    for purpose in purposes:
        dataCategories = purpose.get('dataCategories')
        for dataCategory in dataCategories:
            dataCategory = dataCategory.get('dataCategory')
            category_list.append(dataCategory)
    return list(set(category_list))

In [471]:
df['purposes'] = df['DATA_LINKED_TO_YOU'].apply(extract_purposes)

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [472]:
df

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac..."
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac..."
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Diagnostics, Usage Data, Contact..."
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac..."
5,Wally: Smart Personal Finance,https://apps.apple.com/us/app/wally-smart-pers...,3.8,1600,Budgeting & money management,3.8 • 1.6K Ratings,"{'data': [{'id': '1178011327', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Usage Data, Identifiers, Contact In..."
...,...,...,...,...,...,...,...,...,...,...
197,LazyPay: Loan App & Pay Later,https://apps.apple.com/in/app/lazypay-loan-app...,4.8,116.1K,"Buy now, pay later",4.8 • 116.7K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac..."
198,Zilch: Pay over time,https://apps.apple.com/gb/app/zilch-pay-over-t...,4.2,2.1K,"Buy now, pay later",4.2 • 2.1K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac..."
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1548644102', 'type': 'apps',...",[{'privacyType': 'Datos usados para rastrearte...,"{'privacyType': 'Datos vinculados contigo', 'i...","[Diagnósticos, Datos de uso, IdentificadoADres..."
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]"


In [374]:
def extract_dataCategories(json_data):
    
    dataType_list = []
    purposes = json_data.get('purposes')
    for purpose in purposes:
        dataCategories = purpose.get('dataCategories')
        for dataCategory in dataCategories: 
            dataTypes = dataCategory.get('dataTypes')
            for dataType in dataTypes:
                dataType_list.append(dataType)
   
    return list(set(dataType_list))

In [473]:
df['dataCategories'] = df['DATA_LINKED_TO_YOU'].apply(extract_dataCategories)

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [474]:
df

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras..."
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other User Contact Info, Precise Location, Pr..."
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Diagnostics, Usage Data, Contact...","[Other Diagnostic Data, Crash Data, Product In..."
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras..."
5,Wally: Smart Personal Finance,https://apps.apple.com/us/app/wally-smart-pers...,3.8,1600,Budgeting & money management,3.8 • 1.6K Ratings,"{'data': [{'id': '1178011327', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Usage Data, Identifiers, Contact In...","[Other Diagnostic Data, Contacts, Purchase His..."
...,...,...,...,...,...,...,...,...,...,...,...
197,LazyPay: Loan App & Pay Later,https://apps.apple.com/in/app/lazypay-loan-app...,4.8,116.1K,"Buy now, pay later",4.8 • 116.7K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Customer Supp..."
198,Zilch: Pay over time,https://apps.apple.com/gb/app/zilch-pay-over-t...,4.2,2.1K,"Buy now, pay later",4.2 • 2.1K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Customer Supp..."
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1548644102', 'type': 'apps',...",[{'privacyType': 'Datos usados para rastrearte...,"{'privacyType': 'Datos vinculados contigo', 'i...","[Diagnósticos, Datos de uso, IdentificadoADres...","[Otra información financiera, Dirección física..."
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]","[Phone Number, Email Address, User ID, Name, D..."


In [475]:
df['data_segment_number'] = df['dataCategories'].apply(lambda x: len(x))

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [476]:
df

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras...",11
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other User Contact Info, Precise Location, Pr...",20
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Diagnostics, Usage Data, Contact...","[Other Diagnostic Data, Crash Data, Product In...",9
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras...",13
5,Wally: Smart Personal Finance,https://apps.apple.com/us/app/wally-smart-pers...,3.8,1600,Budgeting & money management,3.8 • 1.6K Ratings,"{'data': [{'id': '1178011327', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Usage Data, Identifiers, Contact In...","[Other Diagnostic Data, Contacts, Purchase His...",13
...,...,...,...,...,...,...,...,...,...,...,...,...
197,LazyPay: Loan App & Pay Later,https://apps.apple.com/in/app/lazypay-loan-app...,4.8,116.1K,"Buy now, pay later",4.8 • 116.7K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Customer Supp...",14
198,Zilch: Pay over time,https://apps.apple.com/gb/app/zilch-pay-over-t...,4.2,2.1K,"Buy now, pay later",4.2 • 2.1K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Customer Supp...",14
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1548644102', 'type': 'apps',...",[{'privacyType': 'Datos usados para rastrearte...,"{'privacyType': 'Datos vinculados contigo', 'i...","[Diagnósticos, Datos de uso, IdentificadoADres...","[Otra información financiera, Dirección física...",15
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]","[Phone Number, Email Address, User ID, Name, D...",6


In [477]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 201
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 192 non-null    object 
 1   url                  192 non-null    object 
 2   stars                192 non-null    float64
 3   ratings              192 non-null    object 
 4   category             192 non-null    object 
 5   scraped_rating       192 non-null    object 
 6   json                 192 non-null    object 
 7   privacyDetails       192 non-null    object 
 8   DATA_LINKED_TO_YOU   192 non-null    object 
 9   purposes             192 non-null    object 
 10  dataCategories       192 non-null    object 
 11  data_segment_number  192 non-null    int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 19.5+ KB


In [413]:
def convert_to_int(value):
    try:
        # Check if the value contains 'K' for thousands
        if 'K' in value:
            return int(float(value.replace('K', '')) * 1000)
        elif 'k' in value:
            return int(float(value.replace('k', '')) * 1000)
        # Check if the value contains 'M' for millions
        elif 'M' in value:
            return int(float(value.replace('M', '')) * 1000000)
        # For numeric values, convert to int
        else:
            return int(value)
    except:
        return value  

In [478]:
df['ratings'] = df['ratings'].apply(convert_to_int)

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [479]:
df

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras...",11
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other User Contact Info, Precise Location, Pr...",20
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Diagnostics, Usage Data, Contact...","[Other Diagnostic Data, Crash Data, Product In...",9
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras...",13
5,Wally: Smart Personal Finance,https://apps.apple.com/us/app/wally-smart-pers...,3.8,1600,Budgeting & money management,3.8 • 1.6K Ratings,"{'data': [{'id': '1178011327', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Usage Data, Identifiers, Contact In...","[Other Diagnostic Data, Contacts, Purchase His...",13
...,...,...,...,...,...,...,...,...,...,...,...,...
197,LazyPay: Loan App & Pay Later,https://apps.apple.com/in/app/lazypay-loan-app...,4.8,116100,"Buy now, pay later",4.8 • 116.7K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Customer Supp...",14
198,Zilch: Pay over time,https://apps.apple.com/gb/app/zilch-pay-over-t...,4.2,2100,"Buy now, pay later",4.2 • 2.1K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Customer Supp...",14
199,Tickmill,https://apps.apple.com/es/app/tickmill/id15486...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1548644102', 'type': 'apps',...",[{'privacyType': 'Datos usados para rastrearte...,"{'privacyType': 'Datos vinculados contigo', 'i...","[Diagnósticos, Datos de uso, IdentificadoADres...","[Otra información financiera, Dirección física...",15
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]","[Phone Number, Email Address, User ID, Name, D...",6


# Analysis

Rank apps across all categories and per category on having the most to least data segments collected

## Rank apps

### All categories

#### Most segments

In [481]:
all_most_segments = df.sort_values(by='data_segment_number', ascending=False)

In [482]:
all_most_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
51,Yahoo Finance: Stocks & News,https://apps.apple.com/us/app/yahoo-finance-st...,4.7,540400,Stock trading & investing,4.7 • 541K Ratings,"{'data': [{'id': '938003185', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",25
50,Robinhood: Investing for All,https://apps.apple.com/us/app/robinhood-invest...,4.2,4200000,Stock trading & investing,4.2 • 4.2M Ratings,"{'data': [{'id': '938003185', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",25
176,Klarna,https://apps.apple.com/us/app/klarna-shop-now-...,4.8,921400,"Buy now, pay later",4.8 • 923.5K Ratings,"{'data': [{'id': '1115120118', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Ident...","[Other User Contact Info, Product Interaction,...",24
180,Sezzle,https://apps.apple.com/us/app/sezzle-buy-now-p...,4.9,130900,"Buy now, pay later",4.9 • 131.8K Ratings,"{'data': [{'id': '283646709', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",23
126,Chime – Mobile Banking,https://apps.apple.com/us/app/chime-mobile-ban...,4.8,600000,Online banking apps,4.8 • 601K Ratings,"{'data': [{'id': '836215269', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Contact I...","[Other User Contact Info, Product Interaction,...",23
...,...,...,...,...,...,...,...,...,...,...,...,...
146,TSB Mobile Banking,https://apps.apple.com/gb/app/tsb-mobile-banki...,4.8,426700,Online banking apps,4.8 • 428.7K Ratings,"{'data': [{'id': '1160720878', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...",[Financial Info],"[Other Financial Info, Payment Info]",2
152,GO2bank,https://apps.apple.com/us/app/go2bank-mobile-b...,4.8,36600,Online banking apps,4.8 • 37K Ratings,"{'data': [{'id': '1534345772', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Financial Info]","[Other Financial Info, Product Interaction]",2
46,Trading・Admirals stocks trade,https://apps.apple.com/gb/app/admirals-trade-s...,3.9,47,Stock trading & investing,3.9 • 47 Ratings,"{'data': [{'id': '1222861799', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...",[Identifiers],[User ID],1
170,Twig - Your Bank of Things,https://apps.apple.com/gb/app/twig-your-bank-o...,3.7,625,Online banking apps,3.6 • 679 Ratings,"{'data': [{'id': '1537416372', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...",[Usage Data],[Product Interaction],1


#### Least segments

In [483]:
all_least_segments = df.sort_values(by='data_segment_number', ascending=True)

In [484]:
all_least_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
47,Plus500 Trading,https://apps.apple.com/il/app/plus500-trading/...,4.1,419,Stock trading & investing,4.1 • 427 Ratings,"{'data': [{'id': '1222861799', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...",[Identifiers],[User ID],1
46,Trading・Admirals stocks trade,https://apps.apple.com/gb/app/admirals-trade-s...,3.9,47,Stock trading & investing,3.9 • 47 Ratings,"{'data': [{'id': '1222861799', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...",[Identifiers],[User ID],1
170,Twig - Your Bank of Things,https://apps.apple.com/gb/app/twig-your-bank-o...,3.7,625,Online banking apps,3.6 • 679 Ratings,"{'data': [{'id': '1537416372', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...",[Usage Data],[Product Interaction],1
61,"RetailMeNot: Coupons, Cashback",https://apps.apple.com/us/app/retailmenot-coup...,4.8,534300,Cash back apps,4.8 • 524.7K Ratings,"{'data': [{'id': '521207075', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers]","[User ID, Product Interaction]",2
142,Metro Bank,https://apps.apple.com/gb/app/metro-bank/id117...,4.8,168100,Online banking apps,4.8 • 169K Ratings,"{'data': [{'id': '1175525592', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Location, Usage Data]","[Precise Location, Product Interaction]",2
...,...,...,...,...,...,...,...,...,...,...,...,...
179,PayPal Pay in 4,https://apps.apple.com/jp/app/paypal-send-shop...,4.4,38100,"Buy now, pay later",4.4 • 38.2K Ratings,"{'data': [{'id': '283646709', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",23
126,Chime – Mobile Banking,https://apps.apple.com/us/app/chime-mobile-ban...,4.8,600000,Online banking apps,4.8 • 601K Ratings,"{'data': [{'id': '836215269', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Contact I...","[Other User Contact Info, Product Interaction,...",23
176,Klarna,https://apps.apple.com/us/app/klarna-shop-now-...,4.8,921400,"Buy now, pay later",4.8 • 923.5K Ratings,"{'data': [{'id': '1115120118', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Ident...","[Other User Contact Info, Product Interaction,...",24
51,Yahoo Finance: Stocks & News,https://apps.apple.com/us/app/yahoo-finance-st...,4.7,540400,Stock trading & investing,4.7 • 541K Ratings,"{'data': [{'id': '938003185', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",25


### Budgeting & money management

#### Most segments

In [486]:
budgeting_most_segments = df[df.category == 'Budgeting & money management']
budgeting_most_segments = budgeting_most_segments.sort_values(by='data_segment_number', ascending=False)
budgeting_most_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other User Contact Info, Precise Location, Pr...",20
14,Albert: Banking on you,https://apps.apple.com/us/app/albert-banking-o...,4.6,155600,Budgeting & money management,4.6 • 156.5K Ratings,"{'data': [{'id': '1057771088', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Oth...",18
20,Venmo,https://apps.apple.com/us/app/venmo/id351727428,4.9,15400000,Budgeting & money management,4.9 • 15.4M Ratings,"{'data': [{'id': '351727428', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Coa...",17
6,PocketGuard: Money & Budgeting,https://apps.apple.com/us/app/pocketguard-mone...,4.7,6700,Budgeting & money management,4.7 • 6.7K Ratings,"{'data': [{'id': '949414211', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Phone Number, Crash Da...",17
13,Marcus by Goldman Sachs®,https://apps.apple.com/us/app/marcus-by-goldma...,4.8,155700,Budgeting & money management,4.8 • 157.2K Ratings,"{'data': [{'id': '1489511701', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Phone Number, Purchase...",17
15,Acorns: Invest Spare Change,https://apps.apple.com/us/app/acorns-invest-sp...,4.7,869900,Budgeting & money management,4.7 • 870.8K Ratings,"{'data': [{'id': '883324671', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Phone Number, Purchase History, Crash Data, C...",15
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras...",13
5,Wally: Smart Personal Finance,https://apps.apple.com/us/app/wally-smart-pers...,3.8,1600,Budgeting & money management,3.8 • 1.6K Ratings,"{'data': [{'id': '1178011327', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Usage Data, Identifiers, Contact In...","[Other Diagnostic Data, Contacts, Purchase His...",13
16,Expensify: Receipts & Expenses,https://apps.apple.com/us/app/expensify-receip...,4.7,129900,Budgeting & money management,4.7 • 130.2K Ratings,"{'data': [{'id': '471713959', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Crash Data, C...",13
17,Intuit Credit Karma,https://apps.apple.com/us/app/intuit-credit-ka...,4.8,6200000,Budgeting & money management,4.8 • 6.3M Ratings,"{'data': [{'id': '471713959', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data 

#### Least segments

In [487]:
budgeting_least_segments = budgeting_most_segments.sort_values(by='data_segment_number', ascending=True)
budgeting_least_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
12,NerdWallet: Manage Your Money,https://apps.apple.com/us/app/nerdwallet-manag...,4.8,102300,Budgeting & money management,4.8 • 102.6K Ratings,"{'data': [{'id': '1174471607', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]","[User ID, Email Address]",2
19,Dollarbird,https://apps.apple.com/us/app/dollarbird-pro/i...,4.7,3300,Budgeting & money management,4.7 • 3.3K Ratings,"{'data': [{'id': '1052884030', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Location, Identifiers]","[User ID, Precise Location]",2
18,FreshBooks Accounting,https://apps.apple.com/us/app/freshbooks-accou...,4.7,10700,Budgeting & money management,4.7 • 10.8K Ratings,"{'data': [{'id': '1052884030', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Location, Identifiers]","[User ID, Precise Location]",2
10,Honeydue: Couples Finance,https://apps.apple.com/us/app/honeydue-couples...,4.5,8200,Budgeting & money management,4.5 • 8.2K Ratings,"{'data': [{'id': '1157633945', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]","[User ID, Device ID, Email Address]",3
25,Wallet - Daily Budget & Profit,https://apps.apple.com/us/app/wallet-daily-bud...,4.6,4200,Budgeting & money management,4.6 • 4.2K Ratings,"{'data': [{'id': '1212024409', 'type': 'apps',...","[{'privacyType': 'Data Not Linked to You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Usage Data, Contact Info]","[Product Interaction, User ID, Device ID, Name...",5
7,Spendee Money & Budget Planner,https://apps.apple.com/us/app/spendee-budget-m...,4.6,4700,Budgeting & money management,4.6 • 4.7K Ratings,"{'data': [{'id': '635861140', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Financial Info, Identifiers, Cont...","[Product Interaction, User ID, Other Financial...",7
23,"Money manager, expense tracker",https://apps.apple.com/us/app/money-manager-ex...,4.9,1500,Budgeting & money management,4.9 • 1.6K Ratings,"{'data': [{'id': '560481810', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Diagnostics, Usage Data, Identifiers, Contact...","[Other Diagnostic Data, Crash Data, Email Addr...",8
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Diagnostics, Usage Data, Contact...","[Other Diagnostic Data, Crash Data, Product In...",9
9,Empower Personal Dashboard™,https://apps.apple.com/us/app/empower-personal...,4.7,39400,Budgeting & money management,4.7 • 39.5K Ratings,"{'data': [{'id': '1449777194', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers, Contact Info, Diagno...","[Phone Number, Crash Data, Email Address, Prod...",10
8,Quicken,https://apps.apple.com/us/app/quicken-simplifi...,4.1,1900,Budgeting & money management,4.1 • 1.9K Ratings,"{'data': [{'id': '1449777194', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers, Contact Info, Diagno...","[Phone Number, Crash Data, Email Address, Prod...",10


### Stock trading & investing

#### Most segments

In [488]:
stock_trading_most_segments = df[df.category == 'Stock trading & investing']
stock_trading_most_segments = stock_trading_most_segments.sort_values(by='data_segment_number', ascending=False)
stock_trading_most_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
51,Yahoo Finance: Stocks & News,https://apps.apple.com/us/app/yahoo-finance-st...,4.7,540400,Stock trading & investing,4.7 • 541K Ratings,"{'data': [{'id': '938003185', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",25
50,Robinhood: Investing for All,https://apps.apple.com/us/app/robinhood-invest...,4.2,4200000,Stock trading & investing,4.2 • 4.2M Ratings,"{'data': [{'id': '938003185', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",25
30,Fidelity Investments,https://apps.apple.com/us/app/fidelity-investm...,4.8,2100000,Stock trading & investing,4.8 • 2.1M Ratings,"{'data': [{'id': '348177453', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Product Interaction, Precise Location, User I...",21
29,J.P. Morgan Mobile,https://apps.apple.com/us/app/j-p-morgan-mobil...,4.8,6200,Stock trading & investing,4.8 • 6.2K Ratings,"{'data': [{'id': '368221343', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",20
41,Cash App,https://apps.apple.com/us/app/square-cash/id71...,4.8,5300000,Stock trading & investing,4.8 • 5.4M Ratings,"{'data': [{'id': '711923939', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Oth...",18
45,eToro: Investing made social,https://apps.apple.com/us/app/etoro-investing-...,4.2,5700,Stock trading & investing,4.2 • 5.8K Ratings,"{'data': [{'id': '711923939', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Oth...",18
44,tastytrade: Invest & Trade,https://apps.apple.com/us/app/tastytrade-inves...,4.8,8700,Stock trading & investing,4.8 • 8.9K Ratings,"{'data': [{'id': '711923939', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Oth...",18
43,IBKR Mobile - Invest Worldwide,https://apps.apple.com/us/app/ibkr-mobile-inve...,4.2,4500,Stock trading & investing,4.2 • 4.6K Ratings,"{'data': [{'id': '711923939', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Oth...",18
42,Wealthfront: Save and Invest,https://apps.apple.com/us/app/wealthfront-save...,4.8,16400,Stock trading & investing,4.8 • 16.4K Ratings,"{'data': [{'id': '711923939', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Oth...",18
40,Acorns: Invest Spare Change,https://apps.apple.com/us/app/acorns-invest-sp...,4.7,869900,Stock trading & investing,4.7 • 870.8K Ratings,"{'data': [{'id': '883324671', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User C

#### Least segments

In [489]:
stock_trading_least_segments = stock_trading_most_segments.sort_values(by='data_segment_number', ascending=True)
stock_trading_least_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
47,Plus500 Trading,https://apps.apple.com/il/app/plus500-trading/...,4.1,419,Stock trading & investing,4.1 • 427 Ratings,"{'data': [{'id': '1222861799', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...",[Identifiers],[User ID],1
46,Trading・Admirals stocks trade,https://apps.apple.com/gb/app/admirals-trade-s...,3.9,47,Stock trading & investing,3.9 • 47 Ratings,"{'data': [{'id': '1222861799', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...",[Identifiers],[User ID],1
37,Firstrade: Invest & Trade,https://apps.apple.com/us/app/firstrade-invest...,4.6,4900,Stock trading & investing,4.6 • 4.9K Ratings,"{'data': [{'id': '405325225', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Financial Info, Identifiers]","[User ID, Device ID, Product Interaction, Paym...",4
31,TD Ameritrade Mobile,https://apps.apple.com/us/app/td-ameritrade-mo...,4.5,119900,Stock trading & investing,4.5 • 119.9K Ratings,"{'data': [{'id': '534659421', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Location, User Content, Financial Info, Ident...","[Coarse Location, Photos or Videos, User ID, O...",5
52,thinkorswim: Trade. Invest.,https://apps.apple.com/us/app/thinkorswim-trad...,4.7,309200,Stock trading & investing,4.7 • 309.6K Ratings,"{'data': [{'id': '299366785', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers]","[Product Interaction, Photos or Videos, User I...",5
200,FXTM Trader - Forex Trading,https://apps.apple.com/me/app/fxtm-trader-fore...,0.0,0,Stock trading & investing,0,"{'data': [{'id': '1265719831', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]","[Phone Number, Email Address, User ID, Name, D...",6
39,Public: Stocks/T-bills/Crypto,https://apps.apple.com/us/app/public-invest-in...,4.7,65099,Stock trading & investing,4.7 • 65.3K Ratings,"{'data': [{'id': '1204112719', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Diagnostics, Usage Data, Contact...","[Other Diagnostic Data, Phone Number, Crash Da...",7
32,Axos Bank®,https://apps.apple.com/us/app/axos-bank/id1396...,4.7,12300,Stock trading & investing,4.7 • 12.4K Ratings,"{'data': [{'id': '1396586421', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Financial Info, Contact Info, Se...","[Phone Number, Sensitive Info, Credit Info, Us...",8
48,IG Trading Platform,https://apps.apple.com/gb/app/ig-trading-platf...,4.6,27300,Stock trading & investing,4.6 • 27.4K Ratings,"{'data': [{'id': '406492428', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Search History, Diagnostics, Usage Data, Iden...","[Other Diagnostic Data, Crash Data, Product In...",9
35,Ally Invest,https://apps.apple.com/us/app/ally-banking-inv...,4.7,64700,Stock trading & investing,4.7 • 65.4K Ratings,"{'data': [{'id': '514374715', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Crash Data, Email Address, Prod...",10


### Cash back apps

#### Most segments

In [490]:
cash_back_most_segments = df[df.category == 'Cash back apps']
cash_back_most_segments = cash_back_most_segments.sort_values(by='data_segment_number', ascending=False)
cash_back_most_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
54,Ibotta: Save & Earn Cash Back,https://apps.apple.com/us/app/ibotta-save-earn...,4.8,1700000,Cash back apps,4.8 • 1.7M Ratings,"{'data': [{'id': '559887125', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Product Interaction, Precise Location, User I...",21
63,Receipt Hog: Shopping Rewards,https://apps.apple.com/us/app/receipt-hog-shop...,4.7,156000,Cash back apps,4.7 • 156.7K Ratings,"{'data': [{'id': '525373618', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Phone Number, Purchase History, Coarse Locati...",16
67,TopCashback: Cash Back & Deals,https://apps.apple.com/us/app/topcashback-cash...,4.7,11800,Cash back apps,4.7 • 11.8K Ratings,"{'data': [{'id': '892405674', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Other Diagnostic Data, Purcha...",16
64,Shopkick: Gift Cards Rewards,https://apps.apple.com/us/app/shopkick-gift-ca...,4.6,79100,Cash back apps,4.6 • 79.2K Ratings,"{'data': [{'id': '525373618', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Phone Number, Purchase History, Coarse Locati...",16
65,CoinOut: Receipts for Rewards,https://apps.apple.com/us/app/coinout-receipts...,4.6,56500,Cash back apps,4.6 • 56.5K Ratings,"{'data': [{'id': '525373618', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Phone Number, Purchase History, Coarse Locati...",16
56,Rakuten: Cash Back & Deals,https://apps.apple.com/us/app/rakuten-cash-bac...,4.8,358600,Cash back apps,4.8 • 358.6K Ratings,"{'data': [{'id': '723134859', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Phone Number, Purchase Histor...",16
58,"Fetch: Have Fun, Save Money",https://apps.apple.com/us/app/fetch-have-fun-s...,4.8,4400000,Cash back apps,4.8 • 4.4M Ratings,"{'data': [{'id': '1182474649', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Coa...",14
55,"PayPal Honey: Coupons, Rewards",https://apps.apple.com/us/app/paypal-honey-cou...,4.4,7300,Cash back apps,4.4 • 7.3K Ratings,"{'data': [{'id': '1358499588', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Purchase History, Crash Data, Coarse Location...",11
68,Capital One Shopping,https://apps.apple.com/us/app/capital-one-shop...,4.8,145600,Cash back apps,4.8 • 153.4K Ratings,"{'data': [{'id': '1089294040', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Contact Info, Identifiers, Purcha...","[Purchase History, Email Address, Product Inte...",11
60,Checkout 51: Cash Back Savings,https://apps.apple.com/us/app/checkout-51-cash...,4.0,6200,Cash back apps,4.0 • 6.2K Ratings,"{'data': [{'id': '1167047511', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers, Contact Info, Purcha...","[Search History, 

#### Least segments

In [491]:
cash_back_least_segments = cash_back_most_segments.sort_values(by='data_segment_number', ascending=True)
cash_back_least_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
61,"RetailMeNot: Coupons, Cashback",https://apps.apple.com/us/app/retailmenot-coup...,4.8,534300,Cash back apps,4.8 • 524.7K Ratings,"{'data': [{'id': '521207075', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers]","[User ID, Product Interaction]",2
201,Freebird: Finance Manager Pro,https://apps.apple.com/gb/app/free-bird-simple...,4.3,9,Cash back apps,4.3 • 9,"{'data': [{'id': '1244075045', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...",[Contact Info],"[Name, Email Address, Phone Number]",3
74,Mobee - Secret Shopping App,https://apps.apple.com/ca/app/mobee-secret-sho...,4.5,149,Cash back apps,4.5 • 150 Ratings,"{'data': [{'id': '506048732', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Data Types, Precise Location, Product I...",7
73,BeFrugal Cash Back & Coupons,https://apps.apple.com/us/app/befrugal-cash-ba...,4.7,6000,Cash back apps,4.7 • 6K Ratings,"{'data': [{'id': '1443673716', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Purchase History, Email Address, Product Inte...",7
72,Drop: Shop Cash Back & Rewards,https://apps.apple.com/ca/app/drop-shop-cash-b...,4.4,12600,Cash back apps,4.4 • 12.6K Ratings,"{'data': [{'id': '1443673716', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Purchase History, Email Address, Product Inte...",7
71,MyPoints: Cashback Rewards,https://apps.apple.com/us/app/mypoints-cashbac...,4.3,5900,Cash back apps,4.3 • 5.9K Ratings,"{'data': [{'id': '1443673716', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Purchase History, Email Address, Product Inte...",7
70,Survey Junkie,https://apps.apple.com/us/app/survey-junkie/id...,4.5,69200,Cash back apps,4.5 • 69.2K Ratings,"{'data': [{'id': '1221433007', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contact Info, Identifiers, Purc...","[Phone Number, Purchase History, Email Address...",7
69,InboxDollars,https://apps.apple.com/us/app/inboxdollars-sur...,4.5,179200,Cash back apps,4.5 • 179.9K Ratings,"{'data': [{'id': '1221433007', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contact Info, Identifiers, Purc...","[Phone Number, Purchase History, Email Address...",7
66,Swagbucks: Surveys for Money,https://apps.apple.com/us/app/swagbucks-survey...,4.4,95300,Cash back apps,4.4 • 95.7K Ratings,"{'data': [{'id': '640439547', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contact Info, Identifiers, Purc...","[Phone Number, Purchase History, Email Address...",7
57,Payce,https://apps.apple.com/us/app/payce/id1484861479,2.8,27,Cash back apps,2.8 • 27 Ratings,"{'data': [{'id': '1484861479', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers, Contact Info, Purcha...","[Purchase History, Coarse Location, Product In...",8


### Coupon apps

#### Most segments

In [492]:
coupon_most_segments = df[df.category == 'Coupon apps']
coupon_most_segments = coupon_most_segments.sort_values(by='data_segment_number', ascending=False)
coupon_most_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
86,Groupon - Local Deals Near Me,https://apps.apple.com/us/app/groupon-local-de...,4.8,452700,Coupon apps,4.8 • 453.7K Ratings,"{'data': [{'id': '352683833', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Product Interaction, Precise Location, User I...",21
80,Ibotta: Save & Earn Cash Back,https://apps.apple.com/us/app/ibotta-save-earn...,4.8,1700000,Coupon apps,4.8 • 1.7M Ratings,"{'data': [{'id': '559887125', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Product Interaction, Precise Location, User I...",21
91,Flipp: Discount Shopping Deals,https://apps.apple.com/us/app/flipp-weekly-sho...,4.8,347800,Coupon apps,4.8 • 350.1K Ratings,"{'data': [{'id': '485357017', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Other User Contact Info, Precise Location, Pr...",21
90,GoodRx: Prescription Saver,https://apps.apple.com/us/app/goodrx-prescript...,4.8,632600,Coupon apps,4.8 • 633.1K Ratings,"{'data': [{'id': '485357017', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Other User Contact Info, Precise Location, Pr...",21
87,Checkout 51: Cash Back Savings,https://apps.apple.com/us/app/checkout-51-cash...,4.0,6200,Coupon apps,4.0 • 6.2K Ratings,"{'data': [{'id': '352683833', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Product Interaction, Precise Location, User I...",21
81,Rakuten: Cash Back & Deals,https://apps.apple.com/us/app/rakuten-cash-bac...,4.8,358600,Coupon apps,4.8 • 358.6K Ratings,"{'data': [{'id': '723134859', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Phone Number, Purchase Histor...",16
76,Target,https://apps.apple.com/us/app/target/id297430070,4.9,5200000,Coupon apps,4.9 • 5.2M Ratings,"{'data': [{'id': '297430070', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Search History, Phone Number, Purchase Histor...",14
77,CouponCabin: Coupon App,https://apps.apple.com/us/app/couponcabin-coup...,4.7,6000,Coupon apps,4.7 • 6K Ratings,"{'data': [{'id': '297430070', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Search History, Phone Number, Purchase Histor...",14
94,Kroger,https://apps.apple.com/us/app/kroger/id403901186,4.8,1200000,Coupon apps,4.8 • 1.2M Ratings,"{'data': [{'id': '1182474649', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Coa...",14
93,CoinOut: Receipts for Rewards,https://apps.apple.com/us/app/coinout-receipts...,4.6,56500,Coupon apps,4.6 • 56.5K Ratings,"{'data': [{'id': '1182474649', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Coa...",14


#### Least segments

In [493]:
coupon_least_segments = coupon_most_segments.sort_values(by='data_segment_number', ascending=True)
coupon_least_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
88,"RetailMeNot: Coupons, Cashback",https://apps.apple.com/us/app/retailmenot-coup...,4.8,524400,Coupon apps,4.8 • 524.7K Ratings,"{'data': [{'id': '521207075', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers]","[User ID, Product Interaction]",2
95,Albertsons Deals & Delivery,https://apps.apple.com/us/app/albertsons-deals...,4.8,296500,Coupon apps,4.8 • 298.2K Ratings,"{'data': [{'id': '542751329', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Diagnostics, Contact Info]","[Other Diagnostic Data, Phone Number, Crash Da...",7
75,The Coupons App,https://apps.apple.com/us/app/the-coupons-app/...,4.3,1400,Coupon apps,4.3 • 1.4K Ratings,"{'data': [{'id': '506048732', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Data Types, Precise Location, Product I...",7
89,GasBuddy: Find & Pay for Gas,https://apps.apple.com/us/app/gasbuddy-find-pa...,4.7,445000,Coupon apps,4.7 • 445.1K Ratings,"{'data': [{'id': '406719683', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers, Contact Info, Locati...","[Coarse Location, Precise Location, Product In...",8
78,Dosh: Find Cash Back Deals,https://apps.apple.com/us/app/dosh-find-cash-b...,4.7,131800,Coupon apps,4.7 • 131.9K Ratings,"{'data': [{'id': '1167047511', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers, Contact Info, Purcha...","[Search History, Phone Number, Purchase Histor...",11
79,"PayPal Honey: Coupons, Rewards",https://apps.apple.com/us/app/honey-coupons-ca...,4.4,7300,Coupon apps,4.4 • 7.3K Ratings,"{'data': [{'id': '1358499588', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Purchase History, Crash Data, Coarse Location...",11
82,Upside: Cash Back - Gas & Food,https://apps.apple.com/us/app/upside-cash-back...,4.8,429500,Coupon apps,4.8 • 433.6K Ratings,"{'data': [{'id': '1099997174', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Other Diagnostic Data, Crash ...",11
83,Shopkick: Gift Cards Rewards,https://apps.apple.com/us/app/shopkick-gift-ca...,4.6,79100,Coupon apps,4.6 • 79.2K Ratings,"{'data': [{'id': '1099997174', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Other Diagnostic Data, Crash ...",11
84,Capital One Shopping Extension,https://apps.apple.com/us/app/capital-one-shop...,4.5,2500,Coupon apps,4.5 • 2.5K Ratings,"{'data': [{'id': '1099997174', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Other Diagnostic Data, Crash ...",11
85,Vipon - Amazon Deals & Coupons,https://apps.apple.com/us/app/vipon-amazon-dea...,3.7,642,Coupon apps,3.7 • 645 Ratings,"{'data': [{'id': '1099997174', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Other Diagnostic Data, Crash ...",11


### Money transfer apps

#### Most segments

In [494]:
money_transfer_most_segments = df[df.category == 'Money transfer apps']
money_transfer_most_segments = money_transfer_most_segments.sort_values(by='data_segment_number', ascending=False)
money_transfer_most_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
99,"PayPal - Send, Shop, Manage",https://apps.apple.com/us/app/paypal-send-shop...,4.8,6200000,Money transfer apps,4.8 • 6.2M Ratings,"{'data': [{'id': '283646709', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",23
100,Cash App,https://apps.apple.com/us/app/cash-app/id71192...,4.8,5300000,Money transfer apps,4.8 • 5.4M Ratings,"{'data': [{'id': '711923939', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Oth...",18
101,Remitly: Send Money & Transfer,https://apps.apple.com/us/app/remitly-send-mon...,4.9,1200000,Money transfer apps,4.9 • 1.2M Ratings,"{'data': [{'id': '674258465', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Phone Number, Purchase History, Crash Data, O...",18
104,Wise (ex TransferWise),https://apps.apple.com/us/app/wise-ex-transfer...,4.6,29700,Money transfer apps,4.6 • 29.8K Ratings,"{'data': [{'id': '612261027', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Contact I...","[Contacts, Phone Number, Purchase History, Cra...",18
106,Cash App,https://apps.apple.com/us/app/square-cash/id71...,4.8,5300000,Money transfer apps,4.8 • 5.4M Ratings,"{'data': [{'id': '711923939', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Oth...",18
97,Venmo,https://apps.apple.com/us/app/venmo/id351727428,4.9,15400000,Money transfer apps,4.9 • 15.4M Ratings,"{'data': [{'id': '351727428', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Coa...",17
112,Payoneer,https://apps.apple.com/us/app/payoneer/id66333...,4.5,2100,Money transfer apps,4.5 • 2.2K Ratings,"{'data': [{'id': '905853115', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Usage Data, Identifiers, Purchases,...","[Other Diagnostic Data, Contacts, Purchase His...",16
111,Pangea: Transfer & Send Money,https://apps.apple.com/us/app/pangea-transfer-...,4.8,3600,Money transfer apps,4.8 • 3.6K Ratings,"{'data': [{'id': '905853115', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Usage Data, Identifiers, Purchases,...","[Other Diagnostic Data, Contacts, Purchase His...",16
115,Revolut - Mobile Finance,https://apps.apple.com/us/app/revolut-mobile-f...,4.7,37000,Money transfer apps,4.7 • 37.2K Ratings,"{'data': [{'id': '718248239', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Identifiers, Purchases, Contact...","[Phone Number, Purchase History, Other User Co...",14
114,BitPay - Bitcoin Wallet & Card,https://apps.apple.com/us/app/bitpay-bitcoin-w...,4.2,2200,Money transfer apps,4.2 • 2.3K Ratings,"{'data': [{'id': '718248239', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Identifiers, Purchases, Contact...","[Phone Number, Purchase History, Other User Co...",14

#### Least segments

In [495]:
money_transfer_least_segments = money_transfer_most_segments.sort_values(by='data_segment_number', ascending=True)
money_transfer_least_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
117,Paysend Money Transfer App,https://apps.apple.com/us/app/paysend-money-tr...,4.6,5400,Money transfer apps,4.6 • 5.6K Ratings,"{'data': [{'id': '1140130413', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]","[Name, Email Address, User ID, Phone Number]",4
105,MoneyGram® Money Transfers App,https://apps.apple.com/us/app/moneygram-money-...,4.9,424800,Money transfer apps,4.9 • 427.1K Ratings,"{'data': [{'id': '867619606', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Financial Info, Contact Info]","[Phone Number, Payment Info, Name, Email Addre...",5
102,WorldRemit Money Transfer,https://apps.apple.com/us/app/worldremit-money...,4.8,172600,Money transfer apps,4.8 • 173.1K Ratings,"{'data': [{'id': '875855935', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Purchase History, Coarse Location, User ID, D...",6
108,OFX Money Transfer,https://apps.apple.com/us/app/ofx-money-transf...,4.9,2400,Money transfer apps,4.9 • 2.4K Ratings,"{'data': [{'id': '1160481993', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Identifiers, Other Data, Locati...","[Coarse Location, Other Data Types, User ID, P...",6
109,WorldRemit Money Transfer,https://apps.apple.com/us/app/worldremit-money...,4.8,172600,Money transfer apps,4.8 • 173.1K Ratings,"{'data': [{'id': '875855935', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Purcha...","[Purchase History, Coarse Location, User ID, D...",6
107,Apple Wallet,https://apps.apple.com/us/app/apple-wallet/id1...,3.1,6000,Money transfer apps,3.1 • 6.1K Ratings,"{'data': [{'id': '1160481993', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Identifiers, Other Data, Locati...","[Coarse Location, Other Data Types, User ID, P...",6
103,Western Union Send Money Now,https://apps.apple.com/us/app/western-union-se...,4.8,633600,Money transfer apps,4.8 • 636.2K Ratings,"{'data': [{'id': '424716908', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Financial Info, Contact Info]","[Phone Number, Other User Contact Info, User I...",8
98,Zelle,https://apps.apple.com/us/app/zelle/id1260755201,4.8,485800,Money transfer apps,4.8 • 487.5K Ratings,"{'data': [{'id': '1260755201', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Contact Info, Identifiers, Diagno...","[Other Diagnostic Data, Phone Number, Precise ...",9
116,"Wirex: Buy, Spend & Sell BTC",https://apps.apple.com/us/app/wirex-buy-spend-...,3.6,584,Money transfer apps,3.6 • 585 Ratings,"{'data': [{'id': '1090004654', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Crash Data, Email Address, Prod...",13
110,Xoom Money Transfer,https://apps.apple.com/us/app/xoom-money-trans...,4.8,1000000,Money transfer apps,4.8 • 1M Ratings,"{'data': [{'id': '529615515', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers, Contact Info, Other ...","[Phone Number, Coarse Location, Other User Con...",14


### Online banking apps

#### Most segments

In [496]:
online_banking_most_segments = df[df.category == 'Online banking apps']
online_banking_most_segments = online_banking_most_segments.sort_values(by='data_segment_number', ascending=False)
online_banking_most_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
126,Chime – Mobile Banking,https://apps.apple.com/us/app/chime-mobile-ban...,4.8,600000,Online banking apps,4.8 • 601K Ratings,"{'data': [{'id': '836215269', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Contact I...","[Other User Contact Info, Product Interaction,...",23
121,Chase Mobile,https://apps.apple.com/us/app/chase-mobile-ban...,4.8,5000000,Online banking apps,4.8 • 5.1M Ratings,"{'data': [{'id': '298867247', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",20
154,Monese: A Banking Alternative,https://apps.apple.com/gb/app/monese-a-banking...,4.8,64400,Online banking apps,4.8 • 64.7K Ratings,"{'data': [{'id': '1102793407', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Sea...",18
165,indi – Smart Banking,https://apps.apple.com/us/app/indi-smart-banki...,4.6,1300,Online banking apps,4.6 • 1.3K Ratings,"{'data': [{'id': '612261027', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Contact I...","[Contacts, Phone Number, Purchase History, Cra...",18
164,Wise,https://apps.apple.com/gb/app/wise/id612261027,4.7,47500,Online banking apps,4.7 • 47.7K Ratings,"{'data': [{'id': '612261027', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Contact I...","[Contacts, Phone Number, Purchase History, Cra...",18
148,Virgin Money Mobile Banking,https://apps.apple.com/gb/app/virgin-money-mob...,4.6,111300,Online banking apps,4.6 • 112.1K Ratings,"{'data': [{'id': '1482269155', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Crash Data, C...",17
138,Starling Bank - Mobile Banking,https://apps.apple.com/gb/app/starling-bank-mo...,4.9,437700,Online banking apps,4.9 • 440.2K Ratings,"{'data': [{'id': '956806430', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Crash Data, Coarse Location, Cu...",16
160,American Airlines Credit Union,https://apps.apple.com/us/app/american-airline...,4.7,13800,Online banking apps,4.7 • 13.8K Ratings,"{'data': [{'id': '1002159038', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Other Diagnostic Data...",16
134,Citi Mobile,https://apps.apple.com/us/app/citi-mobile/id30...,4.9,3900000,Online banking apps,4.9 • 3.6M Ratings,"{'data': [{'id': '301724680', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Crash Data, Emails or...",15
141,NatWest Mobile Banking,https://apps.apple.com/gb/app/natwest-mobile-b...,4.8,368700,Online banking apps,4.8 • 378.2K Ratings,"{'data': [{'id': '334855322', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Coarse L

#### Least segments

In [497]:
online_banking_least_segments = online_banking_most_segments.sort_values(by='data_segment_number', ascending=True)
online_banking_least_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
170,Twig - Your Bank of Things,https://apps.apple.com/gb/app/twig-your-bank-o...,3.7,625,Online banking apps,3.6 • 679 Ratings,"{'data': [{'id': '1537416372', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...",[Usage Data],[Product Interaction],1
152,GO2bank,https://apps.apple.com/us/app/go2bank-mobile-b...,4.8,36600,Online banking apps,4.8 • 37K Ratings,"{'data': [{'id': '1534345772', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Financial Info]","[Other Financial Info, Product Interaction]",2
146,TSB Mobile Banking,https://apps.apple.com/gb/app/tsb-mobile-banki...,4.8,426700,Online banking apps,4.8 • 428.7K Ratings,"{'data': [{'id': '1160720878', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...",[Financial Info],"[Other Financial Info, Payment Info]",2
142,Metro Bank,https://apps.apple.com/gb/app/metro-bank/id117...,4.8,168100,Online banking apps,4.8 • 169K Ratings,"{'data': [{'id': '1175525592', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Location, Usage Data]","[Precise Location, Product Interaction]",2
127,PNC Mobile Banking,https://apps.apple.com/us/app/pnc-mobile-banki...,4.8,1400000,Online banking apps,4.8 • 1.4M Ratings,"{'data': [{'id': '303113127', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers]","[User ID, Device ID, Product Interaction]",3
125,Wells Fargo Mobile,https://apps.apple.com/us/app/wells-fargo-mobi...,4.8,8199999,Online banking apps,4.8 • 8.2M Ratings,"{'data': [{'id': '311548709', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Contact Info]","[Name, User ID, Phone Number, Device ID]",4
157,Amex United Kingdom,https://apps.apple.com/us/app/amex/id362348516,4.9,3300000,Online banking apps,4.9 • 3.3M Ratings,"{'data': [{'id': '362348516', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Diagnostics, Usage Data, Identifiers]","[User ID, Device ID, Product Interaction, Cras...",4
133,Regions Mobile,https://apps.apple.com/us/app/regions-mobile/i...,4.8,394200,Online banking apps,4.8 • 396.8K Ratings,"{'data': [{'id': '417970539', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Location, Diagnostics, Usage Data, Identifiers]","[Crash Data, Precise Location, Product Interac...",5
118,Discover Mobile,https://apps.apple.com/us/app/discover-mobile/...,4.9,4200000,Online banking apps,4.9 • 4.2M Ratings,"{'data': [{'id': '338010821', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Diagnostics, Usage Data, Identi...","[Other Diagnostic Data, Product Interaction, D...",5
173,Huntington Mobile,https://apps.apple.com/us/app/huntington-mobil...,4.8,211400,Online banking apps,4.8 • 211.4K Ratings,"{'data': [{'id': '441599004', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Diagno...","[Other Diagnostic Data, Precise Location, Prod...",6


### Buy now, pay later

#### Most segments

In [498]:
buy_now_pay_later_most_segments = df[df.category == 'Buy now, pay later']
buy_now_pay_later_most_segments = buy_now_pay_later_most_segments.sort_values(by='data_segment_number', ascending=False)
buy_now_pay_later_most_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
176,Klarna,https://apps.apple.com/us/app/klarna-shop-now-...,4.8,921400,"Buy now, pay later",4.8 • 923.5K Ratings,"{'data': [{'id': '1115120118', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Ident...","[Other User Contact Info, Product Interaction,...",24
179,PayPal Pay in 4,https://apps.apple.com/jp/app/paypal-send-shop...,4.4,38100,"Buy now, pay later",4.4 • 38.2K Ratings,"{'data': [{'id': '283646709', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",23
180,Sezzle,https://apps.apple.com/us/app/sezzle-buy-now-p...,4.9,130900,"Buy now, pay later",4.9 • 131.8K Ratings,"{'data': [{'id': '283646709', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Other User Contact Info, Product Interaction,...",23
182,Venmo,https://apps.apple.com/us/app/venmo/id351727428,4.9,15400000,"Buy now, pay later",4.9 • 15.4M Ratings,"{'data': [{'id': '351727428', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Coa...",17
183,Zip,https://apps.apple.com/us/app/zip-buy-now-pay-...,4.9,433400,"Buy now, pay later",4.9 • 434.8K Ratings,"{'data': [{'id': '351727428', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Contacts, Usage Data, Identifie...","[Contacts, Phone Number, Purchase History, Coa...",17
187,Clearpay,https://apps.apple.com/gb/app/clearpay-buy-now...,4.9,49800,"Buy now, pay later",4.9 • 50.5K Ratings,"{'data': [{'id': '1474022186', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Phone Number, Purchase Histor...",16
177,Afterpay,https://apps.apple.com/us/app/afterpay-buy-now...,4.9,734700,"Buy now, pay later",4.9 • 735.1K Ratings,"{'data': [{'id': '1401019110', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Phone Number, Purchase Histor...",16
178,Perpay,https://apps.apple.com/us/app/perpay-shop-and-...,4.6,5200,"Buy now, pay later",4.6 • 5.3K Ratings,"{'data': [{'id': '1401019110', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Search History, Phone Number, Purchase Histor...",16
186,Laybuy,https://apps.apple.com/gb/app/laybuy/id1465232796,4.8,5800,"Buy now, pay later",4.8 • 5.8K Ratings,"{'data': [{'id': '1465232796', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Contact Info, Purch...","[Search History, Phone Number, Purchase Histor...",15
197,LazyPay: Loan App & Pay Later,https://apps.apple.com/in/app/lazypay-loan-app...,4.8,116100,"Buy now, pay later",4.8 • 116.7K Ratings,"{'data': [{'id': '1419990272', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Phone Number, Purchase History, Customer Supp...",14


#### Least segments

In [499]:
buy_now_pay_later_least_segments = buy_now_pay_later_most_segments.sort_values(by='data_segment_number', ascending=True)
buy_now_pay_later_least_segments

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
185,Tabby,https://apps.apple.com/ae/app/tabby-shop-now-p...,4.9,78500,"Buy now, pay later",4.9 • 81.3K Ratings,"{'data': [{'id': '1528022364', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Contact Info]","[Email Address, Phone Number, Product Interact...",3
175,Apple Wallet,https://apps.apple.com/us/app/apple-wallet/id1...,3.1,6000,"Buy now, pay later",3.1 • 6.1K Ratings,"{'data': [{'id': '1160481993', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Identifiers, Other Data, Locati...","[Coarse Location, Other Data Types, User ID, P...",6
196,Split,https://apps.apple.com/us/app/split-enjoy-now-...,3.7,35,"Buy now, pay later",3.6 • 36 Ratings,"{'data': [{'id': '1477987798', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Identifiers, Contact Info, Location...","[Contacts, Phone Number, Coarse Location, Cred...",9
195,Zebit,https://apps.apple.com/us/app/zebit/id1574137193,3.3,78,"Buy now, pay later",3.3 • 78 Ratings,"{'data': [{'id': '1477987798', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Identifiers, Contact Info, Location...","[Contacts, Phone Number, Coarse Location, Cred...",9
194,Nelo,https://apps.apple.com/us/app/nelo/id1477987798,4.4,159,"Buy now, pay later",4.4 • 159 Ratings,"{'data': [{'id': '1477987798', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Identifiers, Contact Info, Location...","[Contacts, Phone Number, Coarse Location, Cred...",9
189,Fingerhut Mobile,https://apps.apple.com/us/app/fingerhut-mobile...,4.8,39300,"Buy now, pay later",4.8 • 40.1K Ratings,"{'data': [{'id': '1500001888', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Diagnostics, Usage Data, Financial Info, Cont...","[Phone Number, Crash Data, Product Interaction...",9
188,ViaBill,https://apps.apple.com/dk/app/viabill/id150000...,4.9,4500,"Buy now, pay later",4.9 • 4.7K Ratings,"{'data': [{'id': '1500001888', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Diagnostics, Usage Data, Financial Info, Cont...","[Phone Number, Crash Data, Product Interaction...",9
184,Four,https://apps.apple.com/us/app/four-pay-later/i...,4.6,9200,"Buy now, pay later",4.6 • 9.3K Ratings,"{'data': [{'id': '1527063669', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Identifiers, Contact Info, Purc...","[Phone Number, Purchase History, Customer Supp...",9
181,Affirm,https://apps.apple.com/us/app/affirm-buy-now-p...,4.9,1300000,"Buy now, pay later",4.9 • 1.3M Ratings,"{'data': [{'id': '967040652', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Financial Info, Contact Info, Pu...","[Phone Number, Purchase History, Credit Info, ...",10
193,LatitudePay,https://apps.apple.com/my/app/latitudepay-my/i...,3.9,227,"Buy now, pay later",4.0 • 239 Ratings,"{'data': [{'id': '995368829', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[Usage Data, Identifiers, Contact Info, Diagno...","[Other Diagnostic Data, Phone Number, Other Us...",13


## Organize tabs and Rankings

### All Apps

Ranking the 20 apps that collect the most data segments
Columns: App name, Number of Data Segments Collected, Data Segment Column (one per data segment) labeled ‘Y’ (yes) or ‘N’ (no).

In [596]:
all_apps = all_most_segments.head(20).reset_index(drop=True)

In [597]:
all_apps.columns

Index(['name', 'url', 'stars', 'ratings', 'category', 'scraped_rating', 'json',
       'privacyDetails', 'DATA_LINKED_TO_YOU', 'purposes', 'dataCategories',
       'data_segment_number'],
      dtype='object')

In [598]:
all_apps = all_apps.drop(columns=['url', 'stars', 'ratings', 'scraped_rating', 
                       'json', 'privacyDetails', 'DATA_LINKED_TO_YOU', 'purposes'])

In [599]:
df_exploded = all_apps.explode('dataCategories')

In [600]:
unique_items = df_exploded['dataCategories'].unique()

In [601]:
unique_items_list = unique_items.tolist()

In [602]:
# function to convert lists to dictionaries

def convert_to_dict(category_list):
    category_dict = {category: 1 if category in category_list else 0 for category in unique_items_list}
    return category_dict

In [603]:
all_apps['dataCategories'] = all_apps['dataCategories'].apply(convert_to_dict)

In [604]:
dummies = pd.DataFrame(all_apps.dataCategories)

In [605]:
# Convert the dictionaries in 'dataCategories' to separate columns with binary values
all_apps = pd.concat([all_apps.drop('dataCategories', axis=1),
                dummies['dataCategories'].apply(lambda x: pd.Series({key: 'Y' if val == 1 else 'N' for key, val in x.items()}))], axis=1)


In [606]:
all_apps

,name,category,data_segment_number,Other User Contact Info,Product Interaction,Precise Location,User ID,Advertising Data,Performance Data,Customer Support,...,Other Usage Data,Other Diagnostic Data,Browsing History,Name,Device ID,Photos or Videos,Audio Data,Emails or Text Messages,Sensitive Info,Health
0,Yahoo Finance: Stocks & News,Stock trading & investing,25,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,N,N,N,N,N
1,Robinhood: Investing for All,Stock trading & investing,25,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,N,N,N,N,N
2,Klarna,"Buy now, pay later",24,Y,Y,N,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,N,N,N,N
3,Sezzle,"Buy now, pay later",23,Y,Y,Y,Y,Y,N,Y,...,Y,N,Y,Y,Y,Y,N,N,N,N
4,Chime – Mobile Banking,Online banking apps,23,Y,Y,Y,Y,N,Y,Y,...,Y,Y,N,Y,N,Y,Y,Y,Y,N
5,PayPal Pay in 4,"Buy now, pay later",23,Y,Y,Y,Y,Y,N,Y,...,Y,N,Y,Y,Y,Y,N,N,N,N
6,"PayPal - Send, Shop, Manage",Money transfer apps,23,Y,Y,Y,Y,Y,N,Y,...,Y,N,Y,Y,Y,Y,N,N,N,N
7,Ibotta: Save & Earn Cash Back,Cash back apps,21,N,Y,Y,Y,Y,Y,Y,...,N,N,Y,Y,Y,Y,N,N,Y,N
8,Checkout 51: Cash Back Savings,Coupon apps,21,N,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,N,N,N,N
9,Fidelity Investments,Stock trading & investing,21,N,Y,Y,Y,Y,Y,Y,...,N,Y,N,Y,Y,Y,Y,N,N,N


### App Cat

Top and bottom app per category for most and least data segments collected.
Columns: App name, Number of Data Segments Collected, Data Segment Column (one per data segment) labeled ‘Y’ (yes) or ‘N’ (no).


In [607]:
app_cat_list = [budgeting_most_segments.head(1), budgeting_least_segments.head(1), 
                stock_trading_most_segments.head(1),stock_trading_least_segments.head(1), 
                cash_back_most_segments.head(1), cash_back_least_segments.head(1),
                coupon_most_segments.head(1), coupon_least_segments.head(1), 
                money_transfer_most_segments.head(1), money_transfer_least_segments.head(1), 
                online_banking_most_segments.head(1), online_banking_least_segments.head(1),
                buy_now_pay_later_most_segments.head(1), buy_now_pay_later_least_segments.head(1)]

In [612]:
app_cat = pd.concat(app_cat_list)

In [613]:
app_cat = app_cat.drop(columns=['url', 'stars', 'ratings', 'scraped_rating', 
                       'json', 'privacyDetails', 'DATA_LINKED_TO_YOU', 'purposes'])

In [614]:
df_exploded = app_cat.explode('dataCategories')
unique_items = df_exploded['dataCategories'].unique()
unique_items_list = unique_items.tolist()
app_cat['dataCategories'] = app_cat['dataCategories'].apply(convert_to_dict)
dummies = pd.DataFrame(app_cat.dataCategories)
# Convert the dictionaries in 'dataCategories' to separate columns with binary values
app_cat = pd.concat([app_cat.drop('dataCategories', axis=1),
                dummies['dataCategories'].apply(lambda x: pd.Series({key: 'Y' if val == 1 else 'N' for key, val in x.items()}))], axis=1)


In [615]:
app_cat

,name,category,data_segment_number,Other User Contact Info,Precise Location,Product Interaction,User ID,Advertising Data,Performance Data,Email Address,...,Device ID,Customer Support,Contacts,Search History,Crash Data,Browsing History,Photos or Videos,Sensitive Info,Audio Data,Emails or Text Messages
2,Mint: Budget & Expense Manager,Budgeting & money management,20,Y,Y,Y,Y,Y,Y,Y,...,Y,N,N,N,N,N,N,N,N,N
12,NerdWallet: Manage Your Money,Budgeting & money management,2,N,N,N,Y,N,N,Y,...,N,N,N,N,N,N,N,N,N,N
51,Yahoo Finance: Stocks & News,Stock trading & investing,25,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,N,N,N,N
47,Plus500 Trading,Stock trading & investing,1,N,N,N,Y,N,N,N,...,N,N,N,N,N,N,N,N,N,N
54,Ibotta: Save & Earn Cash Back,Cash back apps,21,N,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,N,N
61,"RetailMeNot: Coupons, Cashback",Cash back apps,2,N,N,Y,Y,N,N,N,...,N,N,N,N,N,N,N,N,N,N
86,Groupon - Local Deals Near Me,Coupon apps,21,N,Y,Y,Y,Y,Y,Y,...,Y,Y,N,Y,Y,Y,Y,N,N,N
88,"RetailMeNot: Coupons, Cashback",Coupon apps,2,N,N,Y,Y,N,N,N,...,N,N,N,N,N,N,N,N,N,N
99,"PayPal - Send, Shop, Manage",Money transfer apps,23,Y,Y,Y,Y,Y,N,Y,...,Y,Y,Y,Y,N,Y,Y,N,N,N
117,Paysend Money Transfer App,Money transfer apps,4,N,N,N,Y,N,N,Y,...,N,N,N,N,N,N,N,N,N,N


### Banking Apps

In [616]:
online_banking = online_banking_most_segments.drop(columns=['url', 'stars', 'ratings', 'scraped_rating', 
                       'category','json', 'privacyDetails', 'DATA_LINKED_TO_YOU', 'purposes'])

df_exploded = online_banking.explode('dataCategories')
unique_items = df_exploded['dataCategories'].unique()
unique_items_list = unique_items.tolist()
online_banking['dataCategories'] = online_banking['dataCategories'].apply(convert_to_dict)
dummies = pd.DataFrame(online_banking.dataCategories)
# Convert the dictionaries in 'dataCategories' to separate columns with binary values
online_banking = pd.concat([online_banking.drop('dataCategories', axis=1),
                dummies['dataCategories'].apply(lambda x: pd.Series({key: 'Y' if val == 1 else 'N' for key, val in x.items()}))], axis=1)


In [617]:
online_banking

,name,data_segment_number,Other User Contact Info,Product Interaction,Precise Location,User ID,Performance Data,Audio Data,Customer Support,Email Address,...,Payment Info,Other Usage Data,Other Diagnostic Data,Sensitive Info,Name,Advertising Data,Credit Info,Device ID,Search History,Other User Content
126,Chime – Mobile Banking,23,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,N,N,N,N,N
121,Chase Mobile,20,Y,Y,Y,Y,N,N,Y,Y,...,Y,Y,Y,N,Y,Y,Y,Y,N,N
154,Monese: A Banking Alternative,18,N,Y,Y,Y,N,N,Y,Y,...,Y,N,N,N,Y,Y,Y,Y,Y,Y
165,indi – Smart Banking,18,N,Y,N,Y,Y,N,Y,Y,...,Y,N,N,N,Y,N,N,Y,Y,N
164,Wise,18,N,Y,N,Y,Y,N,Y,Y,...,Y,N,N,N,Y,N,N,Y,Y,N
148,Virgin Money Mobile Banking,17,N,Y,Y,Y,Y,N,Y,Y,...,Y,N,N,Y,Y,Y,Y,Y,N,N
138,Starling Bank - Mobile Banking,16,N,Y,N,Y,Y,Y,Y,Y,...,N,N,N,N,Y,N,N,Y,N,Y
160,American Airlines Credit Union,16,Y,Y,N,Y,Y,N,Y,Y,...,Y,Y,Y,N,Y,N,N,Y,N,N
134,Citi Mobile,15,Y,Y,Y,Y,N,N,Y,Y,...,N,Y,N,N,Y,N,N,Y,N,N
141,NatWest Mobile Banking,15,N,Y,N,Y,N,N,N,Y,...,Y,N,N,N,Y,Y,Y,Y,N,Y


### Buy Now, Pay Later

In [618]:
buy_now_pay_later = buy_now_pay_later_most_segments.drop(columns=['url', 'stars', 'ratings', 'scraped_rating', 
                       'category','json', 'privacyDetails', 'DATA_LINKED_TO_YOU', 'purposes'])

df_exploded = buy_now_pay_later.explode('dataCategories')
unique_items = df_exploded['dataCategories'].unique()
unique_items_list = unique_items.tolist()
buy_now_pay_later['dataCategories'] = buy_now_pay_later['dataCategories'].apply(convert_to_dict)
dummies = pd.DataFrame(buy_now_pay_later.dataCategories)
# Convert the dictionaries in 'dataCategories' to separate columns with binary values
buy_now_pay_later = pd.concat([buy_now_pay_later.drop('dataCategories', axis=1),
                dummies['dataCategories'].apply(lambda x: pd.Series({key: 'Y' if val == 1 else 'N' for key, val in x.items()}))], axis=1)


In [619]:
buy_now_pay_later

,name,data_segment_number,Other User Contact Info,Product Interaction,User ID,Advertising Data,Performance Data,Email Address,Customer Support,Physical Address,...,Credit Info,Payment Info,Other Usage Data,Other Diagnostic Data,Browsing History,Name,Device ID,Precise Location,Contacts,Sensitive Info
176,Klarna,24,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,N,N,N
179,PayPal Pay in 4,23,Y,Y,Y,Y,N,Y,Y,Y,...,Y,Y,Y,N,Y,Y,Y,Y,Y,N
180,Sezzle,23,Y,Y,Y,Y,N,Y,Y,Y,...,Y,Y,Y,N,Y,Y,Y,Y,Y,N
182,Venmo,17,N,Y,Y,Y,N,Y,Y,Y,...,Y,Y,N,N,N,Y,Y,Y,Y,N
183,Zip,17,N,Y,Y,Y,N,Y,Y,Y,...,Y,Y,N,N,N,Y,Y,Y,Y,N
187,Clearpay,16,Y,Y,Y,Y,N,Y,Y,Y,...,N,Y,N,N,N,Y,Y,Y,N,N
177,Afterpay,16,Y,Y,Y,Y,N,Y,Y,Y,...,N,Y,N,N,N,Y,Y,Y,N,N
178,Perpay,16,Y,Y,Y,Y,N,Y,Y,Y,...,N,Y,N,N,N,Y,Y,Y,N,N
186,Laybuy,15,N,Y,Y,N,Y,Y,Y,Y,...,Y,Y,N,N,N,Y,N,N,N,N
197,LazyPay: Loan App & Pay Later,14,N,Y,Y,N,N,Y,Y,Y,...,N,Y,Y,N,N,Y,Y,Y,N,N


### Trading & Investment

In [620]:
stock_trading  = stock_trading_most_segments.drop(columns=['url', 'stars', 'ratings', 'scraped_rating', 
                       'category','json', 'privacyDetails', 'DATA_LINKED_TO_YOU', 'purposes'])

df_exploded = stock_trading.explode('dataCategories')
unique_items = df_exploded['dataCategories'].unique()
unique_items_list = unique_items.tolist()
stock_trading['dataCategories'] = stock_trading['dataCategories'].apply(convert_to_dict)
dummies = pd.DataFrame(stock_trading.dataCategories)
# Convert the dictionaries in 'dataCategories' to separate columns with binary values
stock_trading = pd.concat([stock_trading.drop('dataCategories', axis=1),
                dummies['dataCategories'].apply(lambda x: pd.Series({key: 'Y' if val == 1 else 'N' for key, val in x.items()}))], axis=1)


In [621]:
stock_trading

,name,data_segment_number,Other User Contact Info,Product Interaction,Precise Location,User ID,Advertising Data,Performance Data,Customer Support,Email Address,...,Nombre,Atención al cliente,ID de usuario,Número de teléfono,Fotos o vídeos,Datos sensibles,Interacción con el producto,ID del dispositivo,Datos de rendimiento,Sensitive Info
51,Yahoo Finance: Stocks & News,25,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
50,Robinhood: Investing for All,25,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
30,Fidelity Investments,21,N,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
29,J.P. Morgan Mobile,20,Y,Y,Y,Y,Y,N,Y,Y,...,N,N,N,N,N,N,N,N,N,N
41,Cash App,18,Y,Y,N,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
45,eToro: Investing made social,18,Y,Y,N,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
44,tastytrade: Invest & Trade,18,Y,Y,N,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
43,IBKR Mobile - Invest Worldwide,18,Y,Y,N,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
42,Wealthfront: Save and Invest,18,Y,Y,N,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
40,Acorns: Invest Spare Change,15,Y,Y,N,Y,N,N,Y,Y,...,N,N,N,N,N,N,N,N,N,N


### Coupon

In [622]:
coupon = coupon_most_segments.drop(columns=['url', 'stars', 'ratings', 'scraped_rating', 
                       'category','json', 'privacyDetails', 'DATA_LINKED_TO_YOU', 'purposes'])

df_exploded = coupon.explode('dataCategories')
unique_items = df_exploded['dataCategories'].unique()
unique_items_list = unique_items.tolist()
coupon['dataCategories'] = coupon['dataCategories'].apply(convert_to_dict)
dummies = pd.DataFrame(coupon.dataCategories)
# Convert the dictionaries in 'dataCategories' to separate columns with binary values
coupon = pd.concat([coupon.drop('dataCategories', axis=1),
                dummies['dataCategories'].apply(lambda x: pd.Series({key: 'Y' if val == 1 else 'N' for key, val in x.items()}))], axis=1)


In [623]:
coupon

,name,data_segment_number,Product Interaction,Precise Location,User ID,Advertising Data,Performance Data,Customer Support,Email Address,Physical Address,...,Contacts,Other Financial Info,Sensitive Info,Other User Contact Info,Health,Emails or Text Messages,Fitness,Other Data Types,DeviceID,UserID
86,Groupon - Local Deals Near Me,21,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
80,Ibotta: Save & Earn Cash Back,21,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,N,N,N,N,N,N,N
91,Flipp: Discount Shopping Deals,21,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,Y,Y,Y,N,N,N,N,N
90,GoodRx: Prescription Saver,21,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,Y,Y,Y,N,N,N,N,N
87,Checkout 51: Cash Back Savings,21,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
81,Rakuten: Cash Back & Deals,16,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
76,Target,14,Y,Y,Y,Y,N,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
77,CouponCabin: Coupon App,14,Y,Y,Y,Y,N,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
94,Kroger,14,Y,Y,Y,N,Y,N,Y,N,...,Y,N,Y,N,N,Y,N,N,N,N
93,CoinOut: Receipts for Rewards,14,Y,Y,Y,N,Y,N,Y,N,...,Y,N,Y,N,N,Y,N,N,N,N


### Money Management

In [624]:
budgeting = budgeting_most_segments.drop(columns=['url', 'stars', 'ratings', 'scraped_rating', 
                       'category','json', 'privacyDetails', 'DATA_LINKED_TO_YOU', 'purposes'])

df_exploded = budgeting.explode('dataCategories')
unique_items = df_exploded['dataCategories'].unique()
unique_items_list = unique_items.tolist()
budgeting['dataCategories'] = budgeting['dataCategories'].apply(convert_to_dict)
dummies = pd.DataFrame(budgeting.dataCategories)
# Convert the dictionaries in 'dataCategories' to separate columns with binary values
budgeting = pd.concat([budgeting.drop('dataCategories', axis=1),
                dummies['dataCategories'].apply(lambda x: pd.Series({key: 'Y' if val == 1 else 'N' for key, val in x.items()}))], axis=1)


In [625]:
budgeting

,name,data_segment_number,Other User Contact Info,Precise Location,Product Interaction,User ID,Advertising Data,Performance Data,Email Address,Physical Address,...,Credit Info,Payment Info,Other Usage Data,Other Diagnostic Data,Name,Device ID,Contacts,Crash Data,Photos or Videos,Customer Support
2,Mint: Budget & Expense Manager,20,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,N,N,N,N
14,Albert: Banking on you,18,N,N,Y,Y,Y,Y,Y,Y,...,Y,Y,N,Y,Y,Y,Y,Y,Y,N
20,Venmo,17,N,Y,Y,Y,Y,N,Y,Y,...,Y,Y,N,N,Y,Y,Y,N,Y,Y
6,PocketGuard: Money & Budgeting,17,Y,N,Y,Y,Y,Y,Y,Y,...,Y,Y,N,Y,Y,Y,N,Y,N,Y
13,Marcus by Goldman Sachs®,17,N,N,Y,Y,Y,Y,Y,Y,...,Y,Y,N,Y,Y,Y,N,Y,Y,N
15,Acorns: Invest Spare Change,15,Y,N,Y,Y,N,N,Y,Y,...,N,Y,Y,N,Y,Y,N,Y,N,Y
4,Goodbudget Budget Planner,13,N,Y,Y,Y,N,Y,Y,N,...,N,N,Y,Y,N,Y,N,Y,N,N
5,Wally: Smart Personal Finance,13,N,Y,Y,Y,Y,Y,Y,N,...,N,N,Y,Y,N,Y,Y,Y,N,N
16,Expensify: Receipts & Expenses,13,N,Y,Y,Y,N,N,Y,Y,...,N,Y,N,N,Y,N,N,Y,Y,Y
17,Intuit Credit Karma,13,N,Y,Y,Y,N,N,Y,Y,...,N,Y,N,N,Y,N,N,Y,Y,Y


### Raw data

In [627]:
raw_data = pd.concat([df, to_add])

In [628]:
raw_data

,name,url,stars,ratings,category,scraped_rating,json,privacyDetails,DATA_LINKED_TO_YOU,purposes,dataCategories,data_segment_number
0,YNAB: Budgeting & Finance,https://apps.apple.com/us/app/ynab-you-need-a-...,4.8,46400,Budgeting & money management,4.8 • 46.5K Ratings,"{'data': [{'id': '1010865877', 'type': 'apps',...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras...",11.0
2,Mint: Budget & Expense Manager,https://apps.apple.com/us/app/mint-budget-expe...,4.8,783700,Budgeting & money management,4.8 • 784.5K Ratings,"{'data': [{'id': '300238550', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other User Contact Info, Precise Location, Pr...",20.0
3,EveryDollar: Budget Your Money,https://apps.apple.com/us/app/everydollar-budg...,4.7,61500,Budgeting & money management,4.7 • 61.5K Ratings,"{'data': [{'id': '942571931', 'type': 'apps', ...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Identifiers, Diagnostics, Usage Data, Contact...","[Other Diagnostic Data, Crash Data, Product In...",9.0
4,Goodbudget Budget Planner,https://apps.apple.com/us/app/goodbudget-budge...,4.7,12800,Budgeting & money management,4.7 • 12.8K Ratings,"{'data': [{'id': '471112395', 'type': 'apps', ...","[{'privacyType': 'Data Linked to You', 'identi...","{'privacyType': 'Data Linked to You', 'identif...","[User Content, Usage Data, Identifiers, Contac...","[Other Diagnostic Data, Purchase History, Cras...",13.0
5,Wally: Smart Personal Finance,https://apps.apple.com/us/app/wally-smart-pers...,3.8,1600,Budgeting & money management,3.8 • 1.6K Ratings,"{'data': [{'id': '1178011327', 'type': 'apps',...","[{'privacyType': 'Data Used to Track You', 'id...","{'privacyType': 'Data Linked to You', 'identif...","[Contacts, Usage Data, Identifiers, Contact In...","[Other Diagnostic Data, Contacts, Purchase His...",13.0
...,...,...,...,...,...,...,...,...,...,...,...,...
149,Zopa Bank,https://apps.apple.com/gb/app/zopa-bank/id1324...,4.9,20400,Online banking apps,4.9 • 21.1K Ratings,"{'data': [{'id': '1324011914', 'type': 'apps',...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED,NaN,NaN,NaN
169,PenFed Mobile,https://apps.apple.com/us/app/penfed-mobile/id...,4.2,9600,Online banking apps,4.2 • 9.6K Ratings,"{'data': [{'id': '460008350', 'type': 'apps', ...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED,NaN,NaN,NaN
172,Navy Federal Credit Union,https://apps.apple.com/us/app/navy-federal-cre...,4.8,569.5K,Online banking apps,4.9 • 575.4K Ratings,"{'data': [{'id': '370811491', 'type': 'apps', ...","[{'privacyType': 'Data Not Linked to You', 'id...",DATA_NOT_COLLECTED,NaN,NaN,NaN
202,Apple Cash,https://support.apple.com/en-us/HT207886,0.0,0,Money transfer apps,0,NaN,NaN,DATA_NOT_COLLECTED,NaN,NaN,NaN


# Create an Excel file

In [629]:
# Create an Excel writer object

writer = pd.ExcelWriter('Finance Apps That Use The Most Data.xlsx')

In [630]:
# Save each DataFrame to a separate sheet in the Excel file

all_apps.to_excel(writer, sheet_name='All Apps', index=True)
app_cat.to_excel(writer, sheet_name='App Cat', index=True)
online_banking.to_excel(writer, sheet_name='Banking Apps', index=True)
buy_now_pay_later.to_excel(writer, sheet_name='Buy Now, Pay Later', index=True)
stock_trading.to_excel(writer, sheet_name='Trading & Investment', index=True)
coupon.to_excel(writer, sheet_name='Coupon', index=True)
budgeting.to_excel(writer, sheet_name='Money Managment', index=True)
raw_data.to_excel(writer, sheet_name='raw data', index=True)

In [631]:
# Save the Excel file
writer.save()